# FORECASTING
## SCRAPING DATA
Start by defining the current date in the format required by websites for scraping:

In [25]:
from datetime import datetime, timedelta
 
current_date = datetime.now()
current_quarter = round((current_date.month - 1) // 3 + 1)
true_quarter=(current_quarter-1)*3
this_date = datetime(current_date.year, true_quarter-3, 1)
this_date1 = datetime(current_date.year, current_quarter-2, 1)
quarter = this_date.strftime("%Y%m")
quarter1 = this_date1.strftime('%Y%-m')
quarter2 = this_date1.strftime('%Y-Q%-m')
quarter3 = this_date1.strftime('%YQ%-m')
quarter4= this_date.strftime("%Y-%m")
print(quarter1)
print(quarter2)
print(quarter)
print(quarter4)

20211
2021-Q1
202103
2021-03


Now, scrape data for the Czech variables PMI manufacturing, Interest rate 3-month PRIBOR, Nominal wage and Czech GDP: 

In [36]:
#

import requests, urllib
from urllib.request import urlopen
from lxml import etree

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

queries = ["https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=3&p_sort=1&p_des=50&p_sestuid=21430&p_uka=1&p_strid=ACBCB&p_od=200009&p_do=201612&p_lang=CS&p_format=0&p_decsep=%2C",f'https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=3&p_sort=1&p_des=50&p_sestuid=49752&p_uka=1&p_strid=ACBCA&p_od=201703&p_do={quarter}&p_lang=CS&p_format=0&p_decsep=%2C', #PMI_manuf

f'https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=3&p_sort=1&p_des=50&p_sestuid=85&p_uka=6&p_strid=AEAB&p_od=200009&p_do={quarter}&p_lang=CS&p_format=0&p_decsep=%2C',#CNB PRIBOR 3M
f'https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=3&p_sort=1&p_des=50&p_sestuid=21737&p_uka=1&p_strid=ACFA&p_od=200009&p_do={quarter}&p_lang=CS&p_format=0&p_decsep=%2C', #CNB NOM WAGE
f'https://apl.czso.cz/pll/rocenka/rocenka.presmsocas?jmeno_tabulka=CB7&rokod=20003&rokdo={quarter1}&mylang=CZ&ceny=bc&vystup=obrazovka&priznak=CB&typ=2&jak=2&dejarchiv=0'] #CZSO GDP

links = []
df1 = []
d = {}
tables=[]
pmii=[]
yy=[]

for query in queries:
    url = f'{query}'

    ## Make a GET request to fetch the raw HTML content
    headers = {} #'Content-Type': 'text/html',
    html_content = requests.post(url, headers=headers).text

## Parse HTML code for the entire site
    soup = BeautifulSoup(html_content,'html.parser') #lxml#
    gdp = soup.find_all("table")

    #print("Number of tables on site: ",len(gdp))

    # Lets go ahead and scrape first table with HTML code gdp[0]
    table1 = gdp[0]
    body = table1.find_all("tr")
    # Head values (Column names) are the first items of the body list
    head1 = body[0] # 0th item is the header row
    head2=body[1]
    head3=body[2]
    head4=body[4]
    body_rows = body[1:] # All other items becomes the rest of the rows

    if query==queries[0]:
        headings=['YEAR','PMI']

    if query==queries[1]:
        headings=['YEAR','PMI']

    if query==queries[2]:
        headings=['YEAR','R']

    if query==queries[3]:
        headings=['YEAR','Wage']
    if query==queries[4]:
        headings=['YEAR','GDP', 'CONS_TOTAL', 'CONS_HHS', 'CONS_GOV', 'CONS_GOV_IND', 'CONS_GOV_COL', 'CONS_NON', 'CONS_IND', 'THK_TOTAL', 'THFK', 'THK_INVENT','THK_VALUABLES', 'EXPORT_TOTAL', 'EXPORT_GOODS', 'EXPORT_SERVICES', 'IMPORT_TOTAL', 'IMPORT_GOODS', 'IMPORT_SERVICES', 'NX_TOTAL', 'NX_GOODS', 'NX_SERVICES']
        head1 = body[0] # 0th item is the header row
        body_rows = body[5:]

    all_rows = [] # will be a list for list for all rows
    for row_num in range(len(body_rows)): # A row at a time
        row = [] # this will old entries for one row
        for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
            # row_item.text removes the tags from the entries
            # the following regex is to remove \xa0 and \n and comma from row_item.text
            # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
            aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
            #append aa to row - note one row entry is being appended
            row.append(aa)
        # append one row to all_rows
        all_rows.append(row)
    # We can now use the data on all_rows and headings to make a table
    # all_rows becomes our data and headings the column names
    df = pd.DataFrame(data=all_rows,columns=headings)
    if query==queries[0] or query==queries[1]:
        year=df.loc[:,'YEAR']
        pmi0=df.loc[:,'PMI']
        pmii.extend(pmi0) 
        yy.extend(year)

        d = {'YEAR': yy, 'PMI_manuf': pmii}
        df_cnb=pd.DataFrame(data=d)
    if query==queries[2]:
        R=df.loc[:,'R']
        year=df.loc[:,'YEAR']
        d = {'YEAR': year,'PMI_manuf':pmii,'R': R}
        df_R=pd.DataFrame(data=d)

    if query==queries[3]:
        Wage=df.loc[:,'Wage']
        d = {'YEAR': year,'PMI_manuf':pmii,'R': R, 'Wage':Wage}
        df_W=pd.DataFrame(data=d)
    if query==queries[4]:
        df_czso = pd.DataFrame(data=all_rows,columns=headings)
        GDP=df.loc[:,'GDP']
        CONS_TOTAL=df.loc[:,'CONS_TOTAL']
        CONS_HHS=df.loc[:,'CONS_HHS']
        THK_TOTAL=df.loc[:,'THK_TOTAL']
        THFK=df.loc[:,'THFK']
        EXPORT_TOTAL=df.loc[:,'EXPORT_TOTAL']
        IMPORT_TOTAL=df.loc[:,'IMPORT_TOTAL']
        d = {'YEAR': year,'GDP':GDP,'CONS_TOTAL':CONS_TOTAL,'CONS_HHS':CONS_HHS,'THK_TOTAL':THK_TOTAL,'THFK':THFK,'EXPORT_TOTAL':EXPORT_TOTAL,'IMPORT_TOTAL':IMPORT_TOTAL,'PMI_manuf':pmii,'R': R, 'Wage':Wage}
        df_ALL_orig=pd.DataFrame(data=d)
print(((df_ALL_orig)))


       YEAR      GDP CONS_TOTAL CONS_HHS THK_TOTAL    THFK EXPORT_TOTAL  \
0    9/2000   783585     567152   386282    219045  212674       284017   
1   12/2000   786760     564663   384266    221011  217381       302133   
2    3/2001   795830     569811   388254    238471  218736       307690   
3    6/2001   798035     577250   393545    227510  221481       303810   
4    9/2001   801998     585401   398383    222682  224159       307187   
..      ...      ...        ...      ...       ...     ...          ...   
78   3/2020  1293936     854689   594338    360844  346478      1062195   
79   6/2020  1178648     812803   550611    338914  340511       840023   
80   9/2020  1258238     846095   583458    318252  333043      1058856   
81  12/2020  1267513     830068   557832    321105  330609      1122958   
82   3/2021  1263118     828010   556394    364771  333215      1105242   

   IMPORT_TOTAL PMI_manuf    R   Wage  
0        282865       906  534  12963  
1        298252    

Scrape data for the Eurozone GDP:

In [37]:
url = f'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/NAMQ_10_GDP/Q.CP_MEUR.SCA.B1GQ.EA19?startperiod=2000-Q3&endperiod={quarter2}&format=JSON&lang=en'

import requests, json

response = requests.get(url)
data = json.loads(response.text)
multiple_level_data = pd.json_normalize(data['value'])
df = pd.DataFrame.from_dict(multiple_level_data)
df.head(10)
dff=df.T
dyea=dff[0]
dyea=dyea.shift(-20)
GDP=pd.to_numeric(GDP, errors='coerce')
total=[]
YEA=total

import math
#convert dyea to fit datafile
a_value = float('nan')
gg=list(range(len(dyea)))
yea_try=pd.Series(0)
yea_try.repeat=[len(dyea)]
for g in gg:
    if dyea[g]>0:
        yea_try[g]=dyea[g]
delka=len(yea_try)-len(df_ALL_orig)
gg=gg[delka:len(yea_try)]
for g in gg:
    oo=yea_try[g]
    total.append(oo)

R=R[0:len(GDP)]
YEA=YEA[0:len(GDP)]

d = {'GDP':GDP,'CONS_TOTAL':CONS_TOTAL,'CONS_HHS':CONS_HHS,'THK_TOTAL':THK_TOTAL,'THFK':THFK,'EXPORT_TOTAL':EXPORT_TOTAL,'IMPORT_TOTAL':IMPORT_TOTAL,'PMI_MANUF':pmii,'R': R, 'WAGE':Wage, 'YEA':total}
df_ALL=pd.DataFrame(data=d) 
print(df_ALL)


        GDP CONS_TOTAL CONS_HHS THK_TOTAL    THFK EXPORT_TOTAL IMPORT_TOTAL  \
0    783585     567152   386282    219045  212674       284017       282865   
1    786760     564663   384266    221011  217381       302133       298252   
2    795830     569811   388254    238471  218736       307690       314587   
3    798035     577250   393545    227510  221481       303810       305380   
4    801998     585401   398383    222682  224159       307187       307426   
..      ...        ...      ...       ...     ...          ...          ...   
78  1293936     854689   594338    360844  346478      1062195       985248   
79  1178648     812803   550611    338914  340511       840023       819656   
80  1258238     846095   583458    318252  333043      1058856       965150   
81  1267513     830068   557832    321105  330609      1122958      1004026   
82  1263118     828010   556394    364771  333215      1105242      1038577   

   PMI_MANUF    R   WAGE        YEA  
0        906 

Also, scrape Eurostat sentiment indices:

In [28]:
queries=["BS-CCI-BAL","BS-CSMCI-BAL","BS-ICI-BAL","BS-RCI-BAL","BS-SCI-BAL"] 

for query in queries:

    url = f'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/teibs020/M.SA.{query}.CZ?&format=JSON&lang=en'

    import requests, json

    response = requests.get(url)
    data = json.loads(response.text)
    multiple_level_data = pd.json_normalize(data['value'])
    df = pd.DataFrame.from_dict(multiple_level_data)
    dff=df.T
    if query==queries[0]:
        const_conf=multiple_level_data
        const_conf=const_conf.iloc[0, :].values
        const_conf=pd.Series(const_conf)
    if query==queries[1]:
        consumer_conf=multiple_level_data
        consumer_conf=consumer_conf.iloc[0, :].values
        consumer_conf=pd.Series(consumer_conf)
    if query==queries[2]:
        indust_conf=multiple_level_data
        indust_conf=indust_conf.iloc[0, :].values
        indust_conf=pd.Series(indust_conf)
    if query==queries[3]:
        retail_conf=multiple_level_data
        retail_conf=retail_conf.iloc[0, :].values
        retail_conf=pd.Series(retail_conf)
    if query==queries[4]:
        serv_conf=multiple_level_data
        serv_conf=serv_conf.iloc[0, :].values
        serv_conf=pd.Series(serv_conf)
d = {'const_conf':const_conf,'consumer_conf':consumer_conf,'indust_conf':indust_conf,'retail_conf':retail_conf,'serv_conf':serv_conf}
df_sent=pd.DataFrame(data=d)
print(df_sent)

    const_conf  consumer_conf  indust_conf  retail_conf  serv_conf
0        -15.4           -6.6         -8.6         15.1        2.6
1        -12.6           -8.6         -6.8         15.4        0.7
2         -9.4           -8.8         -7.0         13.5       13.1
3         -9.8          -16.4         -5.5         14.9        3.0
4        -11.5          -18.7         -9.9          8.9       -5.0
5         -8.9          -12.6         -5.1         10.6        3.8
6        -11.1          -15.1         -5.7          5.1        3.1
7         -9.6          -16.0         -1.7          4.2        8.1
8         -7.2          -17.1         -3.7          5.2        1.5
9         -8.6           -8.3          3.2         13.1       14.7
10        -4.0           -6.5          5.3         16.7       22.2
11        -5.3           -4.9         10.7         21.1       28.5


Extend the original sentiment dataset for these new values:

In [29]:

CONST=[]
IND=[]
RETAIL=[]
CONSUMER=[]
SERVIC=[]
TOTAL=[]
import pandas as pd
dataset = pd.read_csv('Sentiment1.csv')
headings1=['YEAR', 'MONTH','CONS_INDEX','IND_INDEX','RETAIL_INDEX','CONSUMER_INDEX','SERVIC_INDEX','TOTAL_INDEX'] 
YEAR=dataset.iloc[:, 0].values
MONTH=dataset.iloc[:, 1].values
CONS_INDEX=dataset.iloc[:, 2].values
CONS_INDEX=pd.Series(CONS_INDEX)
IND_INDEX=dataset.iloc[:, 3].values
IND_INDEX=pd.Series(IND_INDEX)
RETAIL_INDEX=dataset.iloc[:, 4].values
RETAIL_INDEX=pd.Series(RETAIL_INDEX)
CONSUMER_INDEX=dataset.iloc[:, 5].values
CONSUMER_INDEX=pd.Series(CONSUMER_INDEX)
SERVIC_INDEX=dataset.iloc[:, 6].values
SERVIC_INDEX=pd.Series(SERVIC_INDEX)
TOTAL_INDEX=dataset.iloc[:, 7].values
TOTAL_INDEX=pd.Series(TOTAL_INDEX)
CONST[0:len(CONS_INDEX)]=(CONS_INDEX)
i=len(CONS_INDEX)+1
j=i+len(const_conf)
CONST[0:len(CONS_INDEX)]=CONS_INDEX
CONST[i:j]=const_conf
CONST=pd.to_numeric(CONST, errors='coerce')

IND[0:len(IND_INDEX)]=IND_INDEX
IND[i:j]=indust_conf
IND=pd.to_numeric(IND, errors='coerce')

RETAIL[0:len(RETAIL_INDEX)]=RETAIL_INDEX
RETAIL[i:j]=retail_conf
RETAIL=pd.to_numeric(RETAIL, errors='coerce')

CONSUMER[0:len(CONSUMER_INDEX)]=CONSUMER_INDEX
CONSUMER[i:j]=consumer_conf
CONSUMER=pd.to_numeric(CONSUMER, errors='coerce')

SERVIC[0:len(SERVIC_INDEX)]=SERVIC_INDEX
SERVIC[i:j]=serv_conf
SERVIC=pd.to_numeric(SERVIC, errors='coerce')

TOTAL=TOTAL_INDEX
TOTAL=pd.to_numeric(TOTAL, errors='coerce')
TOTAL_INDEX=TOTAL

d = {'CONS_INDEX':CONST,'IND_INDEX':IND,'RETAIL_INDEX':RETAIL,'CONSUMER_INDEX':CONSUMER,'SERVIC_INDEX':SERVIC}

df_sent_old=pd.DataFrame(data=d)
print(df_sent_old)



     CONS_INDEX  IND_INDEX  RETAIL_INDEX  CONSUMER_INDEX  SERVIC_INDEX
0         -33.7        8.0          11.3           -21.1           NaN
1         -38.2        8.9           9.9           -21.5           NaN
2         -40.6       15.9          17.2           -22.8           NaN
3         -37.6       15.9          15.8           -20.3           NaN
4         -30.9       22.2          14.2           -18.2           NaN
..          ...        ...           ...             ...           ...
260        -9.6       -1.7           4.2           -16.0           8.1
261        -7.2       -3.7           5.2           -17.1           1.5
262        -8.6        3.2          13.1            -8.3          14.7
263        -4.0        5.3          16.7            -6.5          22.2
264        -5.3       10.7          21.1            -4.9          28.5

[265 rows x 5 columns]


Calculate total sentiment index from the above values:

In [30]:

import statistics
import numpy as np
from statistics import stdev
headings=['CONS_INDEX','CONSUMER_INDEX','IND_INDEX','RETAIL_INDEX']
const=[]
ind=[]
consumer=[]
indust=[]
retail=[]
serv=[]
Zet=[]
ESI_total=[]
total=[]

w=[0.05,0.2,0.4,0.05,0.3]

for head in headings:
    var=head
    var=df_sent_old.loc[:,var]
    average=sum(var)/len(var)
    stdev=np.std(var)
    X=var
    for i in range(len(X)):
        ind.append(i)
        Y=(X[i]-average)/stdev
        if head==headings[0]:
            const.append(Y)
            const_avg=average
            const_stdev=stdev
        if head==headings[1]:
            consumer.append(Y)
            consumer_avg=average
            consumer_stdev=stdev
        if head==headings[2]:
            indust.append(Y)
            indust_avg=average
            indust_stdev=stdev
        if head==headings[3]:
            retail.append(Y)
            retail_avg=average
            retail_stdev=stdev
kon=len(SERVIC_INDEX)
var=SERVIC_INDEX[28:264]
var=pd.to_numeric(var,errors='coerce')
var=pd.Series(var)
serv_avg=sum(var)/len(var)
serv_stdev=np.std(var)
serv_na=[]
for i in range(40):
    serv.append(0)
for i in range(len(SERVIC_INDEX)):
    if i>27:
        Y=(var[i]-serv_avg)/serv_stdev
        serv.append(Y)
for i in range(len(X)):
    ind.append(i)
    if i<28:
        Z=(w[0]*const[i]+w[1]*consumer[i]+w[2]*indust[i]+w[3]*retail[i])/sum(w[0:3])
    if i>=28:
        Z=(w[0]*const[i]+w[1]*consumer[i]+w[2]*indust[i]+w[3]*retail[i]+w[4]*(serv[i]))/sum(w[0:4])
    Zet.append(Z)
Z_avg=sum(Zet)/len(Zet)
Z_stdev=np.std(Zet)
for i in range(len(X)):
    ESI=((Zet[i]-Z_avg)/Z_stdev)*10+100 
    ESI_total.append(ESI)

d = {'CONS_INDEX':CONST,'CONSUMER_INDEX':CONSUMER,'IND_INDEX':IND,'RETAIL_INDEX':RETAIL,'SERV_INDEX':serv, 'TOTAL_INDEX':ESI_total}
df_sent=pd.DataFrame(data=d)
print(df_sent)

     CONS_INDEX  CONSUMER_INDEX  IND_INDEX  RETAIL_INDEX  SERV_INDEX  \
0         -33.7           -21.1        8.0          11.3    0.000000   
1         -38.2           -21.5        8.9           9.9    0.000000   
2         -40.6           -22.8       15.9          17.2    0.000000   
3         -37.6           -20.3       15.9          15.8    0.000000   
4         -30.9           -18.2       22.2          14.2    0.000000   
..          ...             ...        ...           ...         ...   
260        -9.6           -16.0       -1.7           4.2   -1.626193   
261        -7.2           -17.1       -3.7           5.2   -2.484220   
262        -8.6            -8.3        3.2          13.1   -3.163845   
263        -4.0            -6.5        5.3          16.7   -2.416257   
264        -5.3            -4.9       10.7          21.1   -2.475724   

     TOTAL_INDEX  
0      97.908711  
1      97.924978  
2     102.649979  
3     103.545731  
4     108.406145  
..           ...  
26

For the VAR models, scrape OECD data for the variables: Eurozone interest rate, Eurozne HICP, Czech HIC and Czech Employment: 

In [31]:

from pandasdmx import Request
oecd = Request('OECD')

try: #IR EA
    data_response = oecd.data(resource_id='MEI_FIN', key='IR3TIB.EA19.Q', params={'startPeriod': '2000Q3', 'endPeriod':{quarter3}})

except UnicodeDecodeError:
    pass
except KeyError:
    pass
else:
    oecd_data = data_response.data

    df = data_response.write(oecd_data.series, parse_time='True')
    df = data_response.write().unstack().reset_index()
    df_IR_EA=pd.DataFrame(df)
    df_IR_EA = df_IR_EA.sort_values(["TIME_PERIOD"], ascending = (True))
    IR_EA=df_IR_EA.iloc[:,4].values
    
try: #EMPLOYMENT
    data_response = oecd.data(resource_id='QNA', key='CZE.ETONC.PERSA.Q', params={'startPeriod': '2000Q3', 'endPeriod':{quarter3}})

except UnicodeDecodeError:
    pass
except KeyError:
    pass
else:
    oecd_data = data_response.data

    df = data_response.write(oecd_data.series, parse_time='True')
    df = data_response.write().unstack().reset_index()
    df_L=pd.DataFrame(df)
    df_L = df_L.sort_values(["TIME_PERIOD"], ascending = (True))
    L=df_L.iloc[:,5].values
    time=df_L.iloc[:,4].values

try: #CZ_CPI
    data_response = oecd.data(resource_id='PRICES_CPI', key='CZE.CPHPTT01.IXOB.M', params={'startPeriod': '2000-07', 'endPeriod':{quarter4}})

except UnicodeDecodeError:
    pass
except KeyError:
    pass
else:
    oecd_data = data_response.data

    df = data_response.write(oecd_data.series, parse_time='True')
    df = data_response.write().unstack().reset_index()
    df_CZP=pd.DataFrame(df)
    df_CZP = df_CZP.sort_values(["TIME_PERIOD"], ascending = (True))
    CZP_m=df_CZP.iloc[:,5].values
    gg=len(CZP_m)
    CZP=[0]*gg
    for m in range(0,(len(CZP)-1)):
        CZP[m]=sum(CZP_m[(3*m):(2+3*m)])/3
    CZP=CZP[0:len(L)]

try: #EA_CPI
    data_response = oecd.data(resource_id='PRICES_CPI', key='EA19.CPHPTT01.IXOB.M', params={'startPeriod': '2000-07', 'endPeriod':{quarter4}})

except UnicodeDecodeError:
    pass
except KeyError:
    pass
else:
    oecd_data = data_response.data

    df = data_response.write(oecd_data.series, parse_time='True')
    df = data_response.write().unstack().reset_index()
    df_EAP=pd.DataFrame(df)
    df_EAP = df_EAP.sort_values(["TIME_PERIOD"], ascending = (True))
    EAP_m=df_EAP.iloc[:,5].values
    gg=len(EAP_m)
    EAP=[0]*gg
    for m in range(0,(len(EAP)-1)):
        EAP[m]=sum(EAP_m[(3*m):(2+3*m)])/3
    EAP=EAP[0:len(L)]

IR_EA=IR_EA[0:len(L)]
d_oecd = {'time':time,'IR_EA':IR_EA,'L':L,'CZP':CZP,'EAP':EAP}
df_oecd=pd.DataFrame(data=d_oecd)
print(df_oecd)


      time     IR_EA       L        CZP        EAP
0   2000Q3  4.737600  4828.0  49.566667  50.600000
1   2000Q4  5.024167  4842.0  49.633333  50.920000
2   2001Q1  4.745033  4828.0  50.666667  51.030000
3   2001Q2  4.590766  4800.0  51.100000  51.813333
4   2001Q3  4.267833  4825.0  52.166667  51.873333
..     ...       ...     ...        ...        ...
78  2020Q1 -0.405500  5338.0  73.766667  69.643333
79  2020Q2 -0.300667  5286.0  73.900000  70.253333
80  2020Q3 -0.471733  5280.0  74.866667  70.066667
81  2020Q4 -0.522700  5248.0  74.433333  69.983333
82  2021Q1 -0.542467  5249.0  75.333333  70.286667

[83 rows x 5 columns]


For the VAR model, scrape also the Czech national bank dataset for the CZK/EUR exchange rate and the investment:

In [38]:
import requests, urllib
from urllib.request import urlopen
from lxml import etree

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

queries =[f'https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=3&p_sort=1&p_des=50&p_sestuid=55932&p_uka=7&p_strid=AECA&p_od=200009&p_do={quarter}&p_lang=CS&p_format=0&p_decsep=%2C', #CNB exchange rate CZK/EUR
f'https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=3&p_sort=1&p_des=50&p_sestuid=60785&p_uka=7&p_strid=ACCAAB&p_od=200009&p_do={quarter}&p_lang=CS&p_format=0&p_decsep=%2C'#CNB THFK
]

links = []
df1 = []
d = {}
tables=[]
#pmii=[]
yy=[]

for query in queries:
    url = f'{query}'

    ## Make a GET request to fetch the raw HTML content
    headers = {} #'Content-Type': 'text/html',
    html_content = requests.post(url, headers=headers).text

## Parse HTML code for the entire site
    soup = BeautifulSoup(html_content,'html.parser') #lxml#
    gdp = soup.find_all("table")

    #print("Number of tables on site: ",len(gdp))

    # Lets go ahead and scrape first table with HTML code gdp[0]
    table1 = gdp[0]
    body = table1.find_all("tr")
    # Head values (Column names) are the first items of the body list
    head1 = body[0] # 0th item is the header row
    head2=body[1]
    head3=body[2] #vyvoz dovoz saldo
    head4=body[4]
    body_rows = body[1:] # All other items becomes the rest of the rows

    if query==queries[0]:
        headings=['YEAR','NER']

    if query==queries[1]:
        headings=['YEAR','INV']

    all_rows = [] # will be a list for list for all rows
    for row_num in range(len(body_rows)): # A row at a time
        row = [] # this will old entries for one row
        for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
            aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
            #append aa to row - note one row entry is being appended
            row.append(aa)
        # append one row to all_rows
        all_rows.append(row)
    # We can now use the data on all_rowsa and headings to make a table
    # all_rows becomes our data and headings the column names
        df = pd.DataFrame(data=all_rows,columns=headings)
    if query==queries[0]:
        year=df.loc[:,'YEAR']
        NER=df.loc[:,'NER']
        d = {'YEAR': year, 'NER': NER}
        df_NER=pd.DataFrame(data=d)
    if query==queries[1]:
        INV=df.loc[:,'INV']
        INV=pd.Series(INV[0:len(IR_EA)])
        NER=pd.Series(NER[0:len(IR_EA)])
        IR_EA=pd.Series(IR_EA)
        CZP=pd.Series(CZP)
        EAP=pd.Series(EAP)
        L=pd.Series(L)
        year=year[0:len(NER)]
        year=df.loc[:,'YEAR']
        d = {'YEAR': year,'NER':NER,'INV': INV, 'IR_EA':IR_EA,'L':L,'CZP':CZP,'EAP':EAP}
        df_VAR=pd.DataFrame(data=d)
        df_VAR=df_VAR[0:83]
        print(df_VAR[0:83])





          YEAR    NER     INV     IR_EA       L        CZP        EAP
0   III.Q.2000  35425  212469  4.737600  4828.0  49.566667  50.600000
1    IV.Q.2000  34817  217830  5.024167  4842.0  49.633333  50.920000
2     I.Q.2001  34601  218653  4.745033  4828.0  50.666667  51.030000
3    II.Q.2001  33975  221483  4.590766  4800.0  51.100000  51.813333
4   III.Q.2001  34188  224254  4.267833  4825.0  52.166667  51.873333
..         ...    ...     ...       ...     ...        ...        ...
78    I.Q.2020  26575  346478 -0.405500  5338.0  73.766667  69.643333
79   II.Q.2020  26680  340511 -0.300667  5286.0  73.900000  70.253333
80  III.Q.2020  26721  333043 -0.471733  5280.0  74.866667  70.066667
81   IV.Q.2020  26312  330609 -0.522700  5248.0  74.433333  69.983333
82    I.Q.2021  26179  333215 -0.542467  5249.0  75.333333  70.286667

[83 rows x 7 columns]


Combine all the scraped variables and make the final dataset consisting of absolute values:

In [39]:
headings1=['CONS_INDEX','CONSUMER_INDEX','IND_INDEX','RETAIL_INDEX','SERV_INDEX','TOTAL_INDEX']
YoY=[]
years=YEAR
ind = []
CONS_series=[]
IND_series=[]
RETAIL_series=[]
CONSUMER_series=[]
SERVIC_series=[]
TOTAL_series=[]
for i in range(84):
    ind.append(i)
for head in headings1:
    var=head
    MoM=df_sent.loc[:,var]
    for year in ind:
        length=(year)*3
        length_l=length+3
        MoM_part=MoM[length:length_l]  
        MoM_part=pd.to_numeric(MoM_part, errors='coerce')
        MoM_avg=sum(MoM_part)/3
        if head==headings1[0]:
            CONS_series.append(MoM_avg)
        if head==headings1[1]:
            IND_series.append(MoM_avg)
        if head==headings1[2]:
            RETAIL_series.append(MoM_avg)
        if head==headings1[3]:
            CONSUMER_series.append(MoM_avg)
        if head==headings1[4]:
            SERVIC_series.append(MoM_avg)
        if head==headings1[5]:
            TOTAL_series.append(MoM_avg)
d = {'CONS_INDEX':CONS_series,'IND_INDEX':IND_series,'RETAIL_INDEX':RETAIL_series,'CONSUMER_INDEX':CONSUMER_series,'SERVIC_INDEX':SERVIC_series,'TOTAL_INDEX':TOTAL_series}
df_sent_year=pd.DataFrame(data=d)
#print(df_sent_year)

YEAR=df_ALL_orig.iloc[:, 0].values
YEAR=YEAR[0:len(CONS_series[1:])]
GDP=GDP[0:len(CONS_series[1:])]
CONS_TOTAL=CONS_TOTAL[0:len(CONS_series[1:])]
CONS_HHS=CONS_HHS[0:len(CONS_series[1:])]
THK_TOTAL=THK_TOTAL[0:len(CONS_series[1:])]
THFK=THFK[0:len(CONS_series[1:])]
EXPORT_TOTAL=EXPORT_TOTAL[0:len(CONS_series[1:])]
IMPORT_TOTAL=IMPORT_TOTAL[0:len(CONS_series[1:])]
pmii=pmii[0:len(CONS_series[1:])]
R=R[0:len(CONS_series[1:])]
Wage=Wage[0:len(CONS_series[1:])]
YEA=YEA[0:len(CONS_series[1:])]
NER=NER[0:len(CONS_series[1:])]
INV=INV[0:len(CONS_series[1:])]
IR_EA=IR_EA[0:len(CONS_series[1:])]
L=L[0:len(CONS_series[1:])]
CZP=CZP[0:len(CONS_series[1:])]
EAP=EAP[0:len(CONS_series[1:])]

## TOTAL DATASET
d = {'YEAR': YEAR,'GDP':GDP,'CONS_TOTAL':CONS_TOTAL,'CONS_HHS':CONS_HHS,'THK_TOTAL':THK_TOTAL,'THFK':THFK,'EXPORT_TOTAL':EXPORT_TOTAL,'IMPORT_TOTAL':IMPORT_TOTAL,'PMI_MANUF':pmii,'R': R, 'WAGE':Wage, 'YEA':YEA,'CONS_INDEX':CONS_series[1:],'IND_INDEX':IND_series[1:],'RETAIL_INDEX':RETAIL_series[1:],'CONSUMER_INDEX':CONSUMER_series[1:],'SERVIC_INDEX':SERVIC_series[1:],'TOTAL_INDEX':TOTAL_series[1:]}
df_ALL_new=pd.DataFrame(data=d) 

print(df_ALL_new)

d_var = {'YEAR': YEAR,'GDP':GDP,'CONS_TOTAL':CONS_TOTAL,'CONS_HHS':CONS_HHS,'THK_TOTAL':THK_TOTAL,'THFK':THFK,'EXPORT_TOTAL':EXPORT_TOTAL,'IMPORT_TOTAL':IMPORT_TOTAL,'PMI_MANUF':pmii,'R': R, 'WAGE':Wage, 'YEA':YEA,'CONS_INDEX':CONS_series[1:],'IND_INDEX':IND_series[1:],'RETAIL_INDEX':RETAIL_series[1:],'CONSUMER_INDEX':CONSUMER_series[1:],'SERVIC_INDEX':SERVIC_series[1:],'TOTAL_INDEX':TOTAL_series[1:], 'NER':NER,'INV': INV, 'IR_EA':IR_EA,'L':L,'CZP':CZP,'EAP':EAP}
df_ALL_new_var =pd.DataFrame(data=d_var) 
#print(df_ALL_new_var)
df_ALL_new_var.to_csv('GDP_VAR_abs1.csv', index=False)


       YEAR      GDP CONS_TOTAL CONS_HHS THK_TOTAL    THFK EXPORT_TOTAL  \
0    9/2000   783585     567152   386282    219045  212674       284017   
1   12/2000   786760     564663   384266    221011  217381       302133   
2    3/2001   795830     569811   388254    238471  218736       307690   
3    6/2001   798035     577250   393545    227510  221481       303810   
4    9/2001   801998     585401   398383    222682  224159       307187   
..      ...      ...        ...      ...       ...     ...          ...   
78   3/2020  1293936     854689   594338    360844  346478      1062195   
79   6/2020  1178648     812803   550611    338914  340511       840023   
80   9/2020  1258238     846095   583458    318252  333043      1058856   
81  12/2020  1267513     830068   557832    321105  330609      1122958   
82   3/2021  1263118     828010   556394    364771  333215      1105242   

   IMPORT_TOTAL PMI_MANUF    R   WAGE        YEA  CONS_INDEX  IND_INDEX  \
0        282865       90

## DATASET YOY
Calculate the final dataset in year-over-year growth rates:

In [42]:
headings1=['GDP', 'CONS_TOTAL','YEA','EXPORT_TOTAL','IMPORT_TOTAL','PMI_MANUF','WAGE', 'NER','INV', 'L', 'CZP', 'EAP']  # 'THK_TOTAL', 'THFK', 'EXPORT_TOTAL', 
columm=df_ALL_new_var.loc[:,'YEAR']
columm=columm.shift(-4)
columm=columm[0:(len(df_ALL_new)-2)]
dYoY= [columm]#pd.DataFrame([],columns=headings)
dYoY1= []
for head in headings1:
    var=head
    QoQ=df_ALL_new_var.loc[:,var] #df_ALL
    QoQ4=QoQ.shift(-4)
    QoQ=pd.to_numeric(QoQ, errors='coerce')
    QoQ4=pd.to_numeric(QoQ4, errors='coerce')
    YoY=(QoQ4/QoQ-1)*100
    dYoY.append(YoY[0:(len(df_ALL_new)-2)])
IR=df_ALL.loc[:,'R']
IR=pd.to_numeric(IR,errors='coerce')
IR=IR/100
IR=IR.shift(-4)
dYoY.append(IR[0:(len(df_ALL_new)-2)])

IR_EA=df_ALL_new_var.loc[:,'IR_EA']
IR_EA=pd.to_numeric(IR_EA,errors='coerce')
IR_EA=IR_EA
IR_EA=IR_EA.shift(-4)
dYoY.append(IR_EA[0:(len(df_ALL_new)-2)])
#dYoY.append(dyea[0:79])
dfYoY=pd.DataFrame(dYoY)
dfYoY=dfYoY.T
dfYoY_ALL=dfYoY[8:]
LdfYoY=dfYoY_ALL.shift(1)
LdfYoY_ALL=LdfYoY[1:]
dfYoY_ALL=dfYoY[9:]

headings1 = ['CONS_INDEX','IND_INDEX','RETAIL_INDEX','CONSUMER_INDEX','SERVIC_INDEX','TOTAL_INDEX']
dLkon=[]
for head in headings1:
    var=head
    Lkon=df_sent_year.loc[13:(len(df_ALL_new)),var]
    dLkon.append(Lkon)
LdfYoY_kon=pd.DataFrame(dLkon)
LdfYoY_kon=LdfYoY_kon.T

LCONST=LdfYoY_kon['CONS_INDEX']
LIND=LdfYoY_kon['IND_INDEX']
LRETAIL=LdfYoY_kon['RETAIL_INDEX']
LCONSUMER=LdfYoY_kon['CONSUMER_INDEX']
LSERVIC=LdfYoY_kon['SERVIC_INDEX']
LTOTAL=LdfYoY_kon['TOTAL_INDEX']
LY=LdfYoY_ALL['GDP']
LCONS=LdfYoY_ALL['CONS_TOTAL']
LYEA=LdfYoY_ALL['YEA']
LX=LdfYoY_ALL['EXPORT_TOTAL']
LM=LdfYoY_ALL['IMPORT_TOTAL']
LPMI_MANUF=LdfYoY_ALL['PMI_MANUF']
LW=LdfYoY_ALL['WAGE']
LR=LdfYoY_ALL['R']
GDP=dfYoY_ALL['GDP']
CONS_TOTAL=dfYoY_ALL['CONS_TOTAL']
YEA=dfYoY_ALL['YEA']
EXPORT_TOTAL=dfYoY_ALL['EXPORT_TOTAL']
IMPORT_TOTAL=dfYoY_ALL['IMPORT_TOTAL']
PMI_MANUF=dfYoY_ALL['PMI_MANUF']
R=dfYoY_ALL['R']
WAGE=dfYoY_ALL['WAGE']
NER=dfYoY_ALL['NER']
INV=dfYoY_ALL['INV']
IR_EA=dfYoY_ALL['IR_EA']
L=dfYoY_ALL['L']
CZP=dfYoY_ALL['CZP']
EAP=dfYoY_ALL['EAP']
DNX=LX-LM
NXT=EXPORT_TOTAL[0:(len(df_ALL_new)-13)]-IMPORT_TOTAL[0:(len(df_ALL_new)-13)]
COVID=[0]*(len(df_ALL_new)-13)
COVID[(len(COVID)-5):len(COVID)]=[3308, 8646, 58809, 647898, 813671, 134955]
COVID1=COVID[len(COVID)-1]
COVID=pd.Series(COVID)


quarter=[]
years=[]
for k in range(19):
    quarter=(2003+k)
    for i in range(4):
        years.append(quarter)
import numpy
year=years[2:(len(df_ALL_new)-9)]
day=numpy.repeat(1,(len(df_ALL_new)-11))
months=[]
quarters=[0]
for k in range(4):
    quarter=((k+1)*3)
    months.append(quarter)

month1=numpy.tile(months,19)
month=month1[2:(len(df_ALL_new)-9)]

d1={'day':day,'month':month,'year':year,'dy':LY,'dc':LCONS,'dyea':LYEA,'dx':LX,'dm':LM,'dR':LR}
d2={'dConSen_bal':LCONSUMER}
d3={'dPMI_manuf':LPMI_MANUF}
d4={'dInd_conf':LIND,'dConst_conf':LCONST}#'LRETAIL':LRETAIL,,'LSERVIC':LSERVIC,'LTOTAL':LTOTAL
d5={'dw':LW} 
dco={'dnx':DNX}
dco1={'covid': COVID}
d6={'yt':GDP,'ct':CONS_TOTAL,'yeat':YEA,'xt':EXPORT_TOTAL,'mt':IMPORT_TOTAL,'Rt':R} #,'RETAIL_INDEX':RETAIL_series[14:],,'SERVIC_INDEX':SERVIC_series[14:],'TOTAL_INDEX':TOTAL_series[14:]

d7={'ConSen_balt':CONSUMER_series[14:],'PMI_manuft':PMI_MANUF[0:(len(df_ALL_new)-13)],'Ind_conft':IND_series[14:],'Const_conft':CONS_series[14:],'wt':WAGE[0:(len(df_ALL_new)-13)]}
dnx={'nxt':NXT}
dataset0=pd.DataFrame(data=d1)
dataset1=pd.DataFrame(data=d2)
dataset2=pd.DataFrame(data=d3)
dataset3=pd.DataFrame(data=d4)
dataset4=pd.DataFrame(data=d5)
dataset5=pd.DataFrame(data=d6)
dataset6=pd.DataFrame(data=d7)
dataset7=pd.DataFrame(data=dnx)
datasetco=pd.DataFrame(data=dco)
datasetco1=pd.DataFrame(data=dco1)
dataset0.reset_index(drop=True,inplace=True)
dataset1.reset_index(drop=True,inplace=True)
dataset2.reset_index(drop=True,inplace=True)
dataset3.reset_index(drop=True,inplace=True)
dataset4.reset_index(drop=True,inplace=True)
dataset5.reset_index(drop=True,inplace=True)
dataset6.reset_index(drop=True,inplace=True)
dataset7.reset_index(drop=True,inplace=True)
datasetco.reset_index(drop=True,inplace=True)
datasetco1.reset_index(drop=True,inplace=True)
dataset=pd.concat([dataset0[0:(len(df_ALL_new)-13)],dataset1[0:(len(df_ALL_new)-13)],dataset2[0:(len(df_ALL_new)-13)],dataset3[0:(len(df_ALL_new)-13)],dataset4[0:(len(df_ALL_new)-13)],datasetco1[0:(len(datasetco1)-1)],datasetco[0:(len(df_ALL_new)-13)],dataset5[0:(len(df_ALL_new)-13)],dataset6,dataset7],axis=1)
dataset1=dataset.append({'day':day[(len(df_ALL_new)-13)],'month':month[(len(df_ALL_new)-13)], 'year' : year[(len(df_ALL_new)-13)], 'dy':LY[(len(df_ALL_new)-4)],'dc' : LCONS[(len(df_ALL_new)-4)], 'dyea' : LYEA[(len(df_ALL_new)-4)], 'dx' : LX[(len(df_ALL_new)-4)], 'dm' : LM[(len(df_ALL_new)-4)], 'dR' : LR[(len(df_ALL_new)-4)], 'dConSen_bal' : LCONSUMER[(len(df_ALL_new))], 'dPMI_manuf' : LPMI_MANUF[(len(LPMI_MANUF))], 'dInd_conf' : LIND[(len(LIND))], 'dConst_conf' : LCONST[(len(LCONST))], 'dw' : LW[len(LW)], 'covid':COVID1, 'dnx':DNX[len(DNX)], 'yt' : 1.02, 'ct' : 1.02, 'yeat' : 1.02, 'xt' : 1.02, 'mt' : 1.02, 'Rt' : 1.02, 'ConSen_balt' : 1.02, 'PMI_manuft' : 1.02, 'Ind_conft' : 1.02, 'Const_conft' : 1.02, 'wt' : 1.02, 'nxt' : 1.02} , ignore_index=True)

print(dataset1)
dataset1.to_csv('GDP_tree_VARYoY_Yexo1.csv', index=False)

d7={'mae_y':day, 'mae_c':day, 'mae_yea':day, 'mae_x':day, 'mae_m':day, 'mae_r':day, 'mae_con':day, 'mae_pmi':day, 'mae_ind':day, 'mae_const':day, 'mae_w':day, 'mape':day,'acc_y':day, 'acc_c':day, 'acc_yea':day, 'acc_x':day, 'acc_r':day, 'acc_con':day, 'acc_pmi':day, 'acc_ind':day, 'acc_const':day, 'acc_w':day, 'r2_y':day,'r2_c':day,'r2_yea':day,'r2_x':day,'r2_m':day,'r2_r':day,'r2_con':day,'r2_pmi':day,'r2_ind':day,'r2_const':day,'r2_w':day, 'rms_y':day,'rms_c':day,'rms_yea':day,'rms_x':day,'rms_m':day,'rms_r':day,'rms_con':day,'rms_pmi':day,'rms_ind':day,'rms_const':day,'rms_w':day, 'mse':day, 'r2score':day , 'PMI_manuft':day, 'Ind_conft':day, 'Const_conft':day, 'wt':day}
dataset8=pd.DataFrame(data=d7)
dataset8=dataset8[0:(len(df_ALL_new)-12)]
dataset8.reset_index(drop=True,inplace=True)
datasetry=pd.concat([dataset1,dataset8],axis=1)
#print(datasetry)
datasetry.to_csv('GDP_tree_VARYoYry_Y.csv', index=False)
datasetry.to_csv('GDP_tree_VARYoY_Y.csv', index=False)

datasetry.to_csv('GDP_tree_VARYoY_Yrf01.csv', index=False)
datasetry.to_csv('GDP_tree_VARYoYry_Yrf01.csv', index=False)
datasetry.to_csv('GDP_tree_VARYoY1_Yexo1_rf01.csv', index=False)


datasetry.to_csv('GDP_tree_VARYoY_Yxgb01.csv', index=False)
datasetry.to_csv('GDP_tree_VARYoYry_Yxgb01.csv', index=False)
datasetry.to_csv('GDP_tree_VARYoY1_Yexo1_xgb01.csv', index=False)

d8={'NER': NER, 'INV':INV, 'IR_EA':IR_EA, 'L':L, 'CZP': CZP, 'EAP':EAP}
dataset_var=pd.DataFrame(data=d8)
dataset_var=dataset_var[0:(len(df_ALL_new)-12)]
dataset_var.reset_index(drop=True,inplace=True)
datasetry_var=pd.concat([dataset_var,dataset5,dataset6],axis=1)
datasetry_var=datasetry_var[0:(len(datasetry_var)-2)]
d_date={'day':day[1:(len(day)-1)],'month':month[1:(len(month)-1)],'year':year[1:(len(year)-1)]}
data_date=pd.DataFrame(data=d_date)
datasetry_var1=pd.concat([data_date, datasetry_var],axis=1)
#print(datasetry_var1)
datasetry_var1.to_csv('GDP_VAR1.csv', index=False)

    day  month    year       dy        dc      dyea       dx        dm    dR  \
0   1.0    9.0  2003.0  3.85442    5.3531    2.8443  11.4411   10.6424  2.06   
1   1.0   12.0  2003.0  4.12036   5.77238   3.21274  16.7645   15.0024  2.08   
2   1.0    3.0  2004.0  4.01323   1.43881    3.9657  15.1611   15.0612  2.05   
3   1.0    6.0  2004.0  4.14409   1.78055   4.51358   37.561   34.5408  2.33   
4   1.0    9.0  2004.0   4.8072   1.43206   3.78833  35.7319   29.2148  2.72   
..  ...    ...     ...      ...       ...       ...      ...       ...   ...   
66  1.0    3.0  2020.0  -1.4626 -0.580565  -1.54531 -2.98349  -2.53118  1.95   
67  1.0    6.0  2020.0 -10.9421  -6.38865  -12.4122 -24.1937  -18.7091  0.34   
68  1.0    9.0  2020.0 -5.44697  -3.08068  -3.10385 -4.19093  -5.49928  0.34   
69  1.0   12.0  2020.0 -5.34548  -5.13823  -3.38839  3.64573 -0.923147  0.35   
70  1.0    3.0  2021.0 -2.38173  -3.12149  0.333385  4.05265   5.41275  0.36   

    dConSen_bal  ...      yeat       xt

## MODELING PART
### RANDOM FOREST AND XGBOOST MODELS

This section estimates 200 rf/xgb models and selects those with the highest R-squared and lowest rmse to derive the final prediction.

In [ ]:
model='rf' #if you want XGBoost model - type 'xgb' instead of 'rf'
print(model)
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
import numpy as geek

matrix = geek.zeros([20, 20])
matrix_row = pd.Series
vector_zeros=[0]*300
delka=len(vector_zeros)


result_y1=0
result_c1=0
result_yea1=0
result_x1=0
result_m1=0
result_pmi1=0
result_con1=0
result_const1=0
result_ind1=0
result_r1=0
result_w1=0
result_nx1=0

for time in range(1,200):
    for x in range(1,13):

        scenario=x
        #--------------------- EXOGENOUS FORECAST
        # from all the previous loops, only if result_y1 etc. equals 0 meaning the rmse/r2 for all the modelled variables was               below/above threshold value add extra line of the new exogenous prediction to the current dataset:

        if result_y1!=0 and result_c1!=0 and result_yea1!=0 and result_x1!=0 and result_m1!=0 and result_r1!=0 and result_con1!=0 and result_pmi1!=0 and result_ind1!=0 and result_const1!=0 and result_w1!=0 and result_nx1!=0:
            dataset_names_exo=['day',	'month',	'year',	'dy',	'dc',	'dyea',	'dx',	'dm',	'dR',	'dConSen_bal',	'dPMI_manuf',	'dInd_conf',	'dConst_conf',	'dw', 'covid','dnx', 'yt', 'ct', 'yeat', 'xt', 'mt', 'Rt', 'ConSen_balt', 'PMI_manuft', 'Ind_conft', 'Const_conft', 'wt','nxt', 'mae_y',	'mae_c',	'mae_yea',	'mae_x',	'mae_m',	'mae_r',	'mae_con',	'mae_pmi',	'mae_ind',	'mae_const', 'mae_w',	'mape',	'acc_y', 'acc_c', 'acc_yea', 'acc_x', 'acc_r', 'acc_con', 'acc_pmi', 'acc_ind', 'acc_const', 'acc_w',	'r2_y',	'r2_c',	'r2_yea',	'r2_x',	'r2_m',	'r2_r',	'r2_con',	'r2_pmi',	'r2_ind',	'r2_const',	'r2_w',	'rms_y',	'rms_c',	'rms_yea',	'rms_x',	'rms_m',	'rms_r',	'rms_con',	'rms_pmi',	'rms_ind',	'rms_const',	'rms_w',	'mse',	'r2score',	'PMI_manuft',	'Ind_conft',	'Const_conft',	'wt']

            #read the dataset where we will add the new line of the exogeneous prediction
            if model=='xgb':
                dataset_exo = pd.read_csv('GDP_tree_VARYoY1_Yexo1_xgb03.csv') #GDP_tree_VARlags_select.csv
            elif model=='rf':
                dataset_exo = pd.read_csv('GDP_tree_VARYoY1_Yexo1_rf02.csv') #GDP_tree_VARlags_select.csv
            dataset_pd=dataset_exo.iloc[:, 0:78].values 

            dfObj = pd.DataFrame(dataset_pd, columns=dataset_names_exo)

            #replace the 1.02 (ad hoc supplied value) by the actual predicted value from the model available in the matrix
            resy=dfObj.replace({'yt' : {1.02: matrix[vector_zeros[time-2],0]}}) 
            resc=resy.replace({'ct' : {1.02: matrix[vector_zeros[time-2],1]}}) 
            resyea=resc.replace({'yeat' : {1.02: matrix[vector_zeros[time-2],2]}}) 
            resx=resyea.replace({'xt' : {1.02: matrix[vector_zeros[time-2],3]}})
            resm=resx.replace({'mt' : {1.02: matrix[vector_zeros[time-2],4]}})   
            resR=resm.replace({'Rt' : {1.02: matrix[vector_zeros[time-2],5]}}) 
            rescon=resR.replace({'ConSen_balt' : {1.02: matrix[vector_zeros[time-2],6]}}) 
            respmi=rescon.replace({'PMI_manuft' : {1.02: matrix[vector_zeros[time-2],7]}}) 
            resind=respmi.replace({'Ind_conft' : {1.02: matrix[vector_zeros[time-2],8]}}) 
            resconst=resind.replace({'Const_conft' : {1.02: matrix[vector_zeros[time-2],9]}}) 
            resw=resconst.replace({'wt' : {1.02: matrix[vector_zeros[time-2],10]}})  
            resnx=resw.replace({'nxt' : {1.02: matrix[vector_zeros[time-2],11]}})  
            dataset_new=resnx

            #get vector of quarters and years to add to the new prediction line
            quarter=[]
            years=[]
            for k in range(19):
                quarter=(2021+k)
                for i in range(4):
                    years.append(quarter)
            import numpy
            year=years[3:(20)]
            months=[]
            quarters=[0]
            for k in range(4):
                quarter=((k+1)*3)
                months.append(quarter)

            month=numpy.tile(months,19)
            month=month[2:(18)]

            quarter=[]
            years=[]
            for k in range(19):
                quarter=(2021+k)
                for i in range(4):
                    years.append(quarter)
            import numpy
            year=years[1:20]
            months=[]
            quarters=[0]
            for k in range(4):
                quarter=((k+1)*3)
                months.append(quarter)
            print('year',year)
            month=numpy.tile(months,19)
            month=month[1:18]

            #write a new dataset line with the predicted values as available in the "matrix" and for the future add ad hoc values              1.02 that will be later replaced by the actual predicted values
            dataset_new=dataset_new.append({'day' : 1, 'month' : month[vector_zeros[time-2]], 'year' : year[vector_zeros[time-2]], 'dy' : matrix[vector_zeros[time-2],0], 'dc' : matrix[vector_zeros[time-2],1], 'dyea' : matrix[vector_zeros[time-2],2], 'dx' : matrix[vector_zeros[time-2],3], 'dm' : matrix[vector_zeros[time-2],4], 'dR' : matrix[vector_zeros[time-2],5], 'dConSen_bal' : matrix[vector_zeros[time-2],6], 'dPMI_manuf' : matrix[vector_zeros[time-2],7], 'dInd_conf' : matrix[vector_zeros[time-2],8], 'dConst_conf' : matrix[vector_zeros[time-2],9],'covid':0, 'dw' : matrix[vector_zeros[time-2],10], 'dnx' : matrix[vector_zeros[time-2],11], 'yt' : 1.02, 'ct' : 1.02, 'yeat' : 1.02, 'xt' : 1.02, 'mt' : 1.02, 'Rt' : 1.02, 'ConSen_balt' : 1.02, 'PMI_manuft' : 1.02, 'Ind_conft' : 1.02, 'Const_conft' : 1.02, 'wt' : 1.02, 'nxt' : 1.02, 'r2_y' : r2_y, 'r2_c' : r2_c,'r2_yea' : r2_yea, 'r2_x' : r2_x, 'r2_r':r2_r, 'r2_pmi':r2_pmi,'r2_ind':r2_ind,'r2_con':r2_con,'r2_const':r2_const,'r2_w':r2_w} , ignore_index=True)
            
            #save this dataset as a new csv file
            if model=='xgb':
                dataset_new.to_csv('GDP_tree_VARYoY1_Yexo1_xgb03.csv', index=False) #GDP_tree_VARlags_select.csv

            if model=='rf':
                dataset_new.to_csv('GDP_tree_VARYoY1_Yexo1_rf01.csv', index=False) #GDP_tree_VARlags_select.csv
            
        if model=='xgb':
            dataset = pd.read_csv('GDP_tree_VARYoY_Yxgb01.csv') #GDP_tree_VARYoY_Yxgb1
        elif model=='rf':
            dataset = pd.read_csv('GDP_tree_VARYoY_Yrf01.csv') #GDP_tree_VARlags_select.csv

        #select values for the variables used in the model
        
        #1=y
        #for the GDP "y", read values with the net export variable as features
        hh=dataset.head()
        X=dataset.iloc[:, 0:15].values
        X_before = dataset.iloc[:, 0:6].values #25
        X_others=dataset.iloc[:, 8:16].values
        X=np.concatenate((X_before,X_others), axis=1) #features
        y = dataset.iloc[:, 16].values

        #2=c
        #for the model estimating future consumption "c"
        yc=dataset.iloc[:, 17].values
        Xc=dataset.iloc[:, 0:15].values #features

        #3=yea
        #for the model estimating future GDP of the Eurozone "yea"
        yea=dataset.iloc[:, 18].values
        Xea=dataset.iloc[:, 0:15].values

        #4=x
        #for the model estimating future export "x"
        yx=dataset.iloc[:, 19].values
        Xx=dataset.iloc[:, 0:15].values

        #5=m
        #for the model estimating future import "m"
        ym=dataset.iloc[:, 20].values
        Xm=dataset.iloc[:, 0:15].values

        #6=R
        #for the model estimating future interest rate "R"
        yR=dataset.iloc[:, 21].values
        XR=dataset.iloc[:, 0:15].values

        #7=ConSen_bal
        #for the model estimating future Consumers' index "ConSen_bal"
        yCon=dataset.iloc[:, 22].values
        XCon=dataset.iloc[:, 0:15].values

        #8=PMI_manuf
        #for the model estimating future PMI index "PMI_manuf"
        yPMI_manuf=dataset.iloc[:, 23].values
        XPMI_manuf=dataset.iloc[:, 0:15].values

        #9=Ind_conf
        #for the model estimating future industrial confidence "Ind_conf"
        yInd_conf=dataset.iloc[:, 24].values
        XInd_conf=dataset.iloc[:, 0:15].values

        #10=Const_conf
        #for the model estimating future construction confidence "Const_conf"
        yConst_conf=dataset.iloc[:, 25].values
        XConst_conf=dataset.iloc[:, 0:15].values

        #11=w
        #for the model estimating future wage "w"
        yw=dataset.iloc[:, 26].values
        Xw=dataset.iloc[:, 0:15].values

        #12=nx
        #for the model estimating future net export "nx"
        ynx=dataset.iloc[:, 27].values
        Xnx = dataset.iloc[:, 0:6].values #25
        Xnx_others=dataset.iloc[:, 8:16].values
        Xnx=np.concatenate((Xnx,Xnx_others), axis=1)
        
        #in the first loop estimate the model for GDP
        if scenario==1:
            name='dy'
            features=X
            labels=y

        #in the second loop estimate the model for consumption
        elif scenario==2:
            name='dc'
            features=Xc
            labels=yc
        
        #in the third loop estimate the model for GDP EA
        elif scenario==3:
            name='dyea'
            features=Xea
            labels=yea
        
        #in the fourth loop estimate the model for export
        elif scenario==4:
            name='dx'
            features=Xx
            labels=yx

        #in the fifth loop estimate the model for import
        elif scenario==5:
            name='dm'
            features=Xm
            labels=ym

        #in the sixth loop estimate the model for interest rate
        elif scenario==6:
            name='dR'
            features=XR
            labels=yR

        #in the seventh loop estimate the model for consumers' sentiment index
        elif scenario==7:
            name='dConSen_bal'
            features=XCon
            labels=yCon
        
        #in the eigth loop estimate the model for PMI manufacturing
        elif scenario==8:
            name='dPMI_manuf'
            features=XPMI_manuf
            labels=yPMI_manuf

        #in the ninth loop estimate the model for industrial confidence
        elif scenario==9:
            name='dInd_conf'
            features=XInd_conf
            labels=yInd_conf

        #in the tenth loop estimate the model for construction confidence
        elif scenario==10:
            name='dConst_conf'
            features=XConst_conf
            labels=yConst_conf

        #in the eleventh loop estimate the model for wage
        elif scenario==11:
            name='dw'
            features=Xw
            labels=yw  

        #in the twelfth loop estimate the model for net export
        elif scenario==12:
            name='dnx'
            features=Xnx
            labels=ynx   


        from sklearn.model_selection import train_test_split

        # Split the data into training and testing sets, for exogenous predicitions train only on past values prior COVID 0:65
        train_features, test_features, train_labels, test_labels = train_test_split(features[0:70], labels[0:70], test_size = 

        #make exogenous forecast from the randomly selected values and always add future row to get future predictions
        if scenario==12 or scenario==1: #for the model of GDP and Net export estimate the forecast with respect to the past      values of the Net export variable and thus regardless Export and Import variables
            future_row1=dataset.iloc[(len(dataset)-1):(len(dataset)), 0:6].values
            future_row2=dataset.iloc[(len(dataset)-1):(len(dataset)), 8:16].values
            future_row=np.concatenate((future_row1,future_row2), axis=1)
            future_features=test_features
            future=geek.zeros([len(test_features)+2, 14])

        elif scenario!=12 and scenario!=1: #estimate the remaining models with regards to Export and Import variables
            future_row=dataset.iloc[(len(dataset)-1):(len(dataset)), 0:15].values
            future_features=test_features
            future=geek.zeros([len(test_features)+2, 15])
        future_labels=future[0:len(test_labels), 1]*0

        future[0:len(test_features),:]=future_features
        future[len(test_features),:]=future_row
        future=future[1:(len(future)-1),:]

        future_labels[0:(len(test_labels)-1)]=test_labels[1:(len(test_labels))]
        future_labels[(len(future_labels)-1):len(future_labels)]=labels[len(labels)-1]

        # Saving feature names for later use
        feature_list=['day', 'month', 'year', 'dy', 'dc', 'dyea', 'dx',	'dR', 'dConSen_bal', 'dPMI_manuf', 'dInd_conf', 'dConst_conf', 'dw']
        baseline_preds = test_features[:, feature_list.index('dyea')]
        print(baseline_preds)

        # Baseline errors, and display average baseline error
        baseline_errors = abs(baseline_preds - test_labels)
        print('Average baseline error: ', round(np.mean(baseline_errors), 2))

        #If we can’t beat an average error of 0.49 degrees, then we need to rethink our approach.

        # Import the model we are using
        from sklearn.ensemble import RandomForestRegressor
        # Instantiate model with 1000 decision trees
        rf = RandomForestRegressor(n_estimators= 1000) #, random_state=42
        xgb = XGBRegressor(importance_type='gain',objective='reg:squarederror',n_estimators= 1000)

        #xgb=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        #importance_type='gain', learning_rate=0.1, max_delta_step=0,
        #min_child_weight=1, n_estimators=1000, objective='reg:squarederror', random_state=0)
        # Train the model on training data
        rf.fit(train_features, train_labels)
        xgb.fit(train_features, train_labels)

        # # Use the forest's predict method on the test data
        prediction_rf = rf.predict(test_features)
        prediction_xgb=xgb.predict(test_features)
        prediction_xgb_exo=xgb.predict(future)
        prediction_rf_exo=rf.predict(future)

        if model=='xgb':
            predictions=prediction_xgb 
            predictions_exo=prediction_xgb_exo
        if model=='rf':
            predictions=prediction_rf
            predictions_exo=prediction_rf_exo
        # Calculate the absolute errors
        errors = abs(predictions - test_labels)
        # Print out the mean absolute error (mae)
        mae=(round(np.mean(errors), 2))
        print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.') #the average distance btw predicted and actual value

        # # Calculate mean absolute percentage error (MAPE)
        mape = 100 * abs(errors /(test_labels))
        # Calculate and display accuracy
        accuracy = 100 - np.mean(mape)
        print('Accuracy:', round(accuracy, 2), '%.')
        from sklearn.metrics import mean_squared_error
        from math import sqrt
        from sklearn.metrics import r2_score

        rms = sqrt(mean_squared_error(test_labels, predictions))
        print('RMSE', rms) #tip: measure for testing and training data, compare difference

        # from sklearn.metrics import mean_squared_error, r2_score
        model_score =  metrics.r2_score(test_labels, predictions)
        print('coefficient of determination R^2 of the prediction.: ',model_score)
        r2=(model_score)

        # measure the accuracy of the model for all the test values without the last value that enters the endogenous or        exogenous prediction
        r2_endo=predictions*0
        rms_endo = predictions*0
        errors_endo = predictions*0
        mae_endo = predictions*0
        for per in range(0,len(predictions)):
            test_labels_endo=test_labels[0:(len(test_labels)-1)]*0
            predictions_endo=predictions[0:(len(predictions)-1)]*0
                
            if per>0:
                test_labels_endo[0:(per-1)]=test_labels[0:(per-1)]
                predictions_endo[0:(per-1)]=predictions[0:(per-1)]
                test_labels_endo[(per):len(test_labels_endo)]=test_labels[(per+1):len(test_labels)]
                predictions_endo[per:len(predictions_endo)]=predictions[(per+1):len(predictions)]

            if per==0:
                test_labels_endo[0:len(test_labels_endo)]=test_labels[1:len(test_labels)]
                predictions_endo[0:len(predictions_endo)]=predictions[1:len(predictions)]

            r2_endo[per]=metrics.r2_score(test_labels_endo, predictions_endo)
            rms_endo[per]=sqrt(mean_squared_error(test_labels_endo, predictions_endo))
            errors_endo = abs(predictions_endo - test_labels_endo)
            mae_endo[per]=(round(np.mean(errors_endo), 2))
        print('r2_endo',r2_endo)

        model_score_future= metrics.r2_score(future_labels[0:(len(future_labels)-1)], predictions_exo[0:(len(predictions_exo)-1)])
        r2_future=model_score_future
        print('r2_future', r2_future)
        rms_future=sqrt(mean_squared_error(future_labels[0:(len(future_labels)-1)], predictions_exo[0:(len(predictions_exo)-1)]))

        print("Mean squared error: %.2f"% mean_squared_error(test_labels, predictions))
        mse=(mean_squared_error(test_labels, predictions))
        # Explained variance score: 1 is perfect prediction
        print('Test Variance score: %.2f' % r2_score(test_labels, predictions))
        r2score=(r2_score(test_labels, predictions))

        score=rf.score(test_features, test_labels)

        # #--------------------Variable importance
        # Get numerical feature importances
        importances = list(rf.feature_importances_)
        # List of tuples with variable and importance
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
        # Sort the feature importances by most important first
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
        # Print out the feature and importances 
        [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

    #------------------VISUALIZATION
    #   Visualize the variable importance
        import matplotlib.pyplot as plt

        #matplotlib inline
        # Set the style
        plt.style.use('fivethirtyeight')
        # list of x locations for plotting
        x_values = list(range(len(importances)))
        # Make a bar chart
        plt.bar(x_values, importances, orientation = 'vertical')
        # Tick labels for x axis
        plt.xticks(x_values, feature_list, rotation='vertical')
        # Axis labels and title
        plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances')
        fig1 = plt.gcf()
        #plt.show()

        # #Write graph to a png file
        var_name = 'Forest_importance_Y'
        var_name += name
        var_name += '.png'
        # fig1.savefig(var_name, bbox_inches='tight', dpi=100)

        #---------------------ENDOGENOUS FORECAST
        #---------------------Append forecast to original dataset
    
        dataset_names=['day',	'month',	'year',	'dy',	'dc',	'dyea',	'dx',	'dm',	'dR',	'dConSen_bal',	'dPMI_manuf',	'dInd_conf',	'dConst_conf',	'dw', 'covid','dnx', 'yt', 'ct', 'yeat', 'xt', 'mt', 'Rt', 'ConSen_balt', 'PMI_manuft', 'Ind_conft', 'Const_conft', 'wt','nxt', 'mae_y',	'mae_c',	'mae_yea',	'mae_x',	'mae_m',	'mae_r',	'mae_con',	'mae_pmi',	'mae_ind',	'mae_const', 'mae_w',	'mape',	'acc_y', 'acc_c', 'acc_yea', 'acc_x', 'acc_r', 'acc_con', 'acc_pmi', 'acc_ind', 'acc_const', 'acc_w',	'r2_y',	'r2_c',	'r2_yea',	'r2_x',	'r2_m',	'r2_r',	'r2_con',	'r2_pmi',	'r2_ind',	'r2_const',	'r2_w',	'rms_y',	'rms_c',	'rms_yea',	'rms_x',	'rms_m',	'rms_r',	'rms_con',	'rms_pmi',	'rms_ind',	'rms_const',	'rms_w',	'mse',	'r2score',	'PMI_manuft',	'Ind_conft',	'Const_conft',	'wt'] 

        #read the dataset where you want to add the endogenous forecast
        if model=='xgb':
            dataset1 = pd.read_csv('GDP_tree_VARYoYry_Yxgb03.csv')
        if model=='rf':
            dataset1 = pd.read_csv('GDP_tree_VARYoYry_Yrf01.csv')
        dataset_pd=dataset1.iloc[:, 0:78].values #29
        dfObj = pd.DataFrame(dataset_pd, columns=dataset_names)

        #this line is important for the exogenous forecast
        #only if all the variables are now predicted for the future add 1 to the vector called vector_zeros so you can add new             line to the "matrix" filled with future values.
        if result_y1!=0 and result_c1!=0 and result_yea1!=0 and result_x1!=0 and result_m1!=0 and result_r1!=0 and result_con1!=0 and result_pmi1!=0 and result_ind1!=0 and result_const1!=0 and result_w1!=0 and result_nx1!=0:
            vector_zeros[time-1]=vector_zeros[time-2]+1
            for k in range(time,delka-1):
                vector_zeros[k]=vector_zeros[k-1]
            result_y1=0
            result_c1=0
            result_yea1=0
            result_x1=0
            result_m1=0
            result_pmi1=0
            result_con1=0
            result_const1=0
            result_ind1=0
            result_r1=0
            result_w1=0
            result_nx1=0

        if scenario==1: 

            #this accuracy measure is set for the exogenous forecast:
            if r2_future>0.9:
                if model=='xgb':
                    result_y1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_y1=prediction_rf_exo[len(prediction_rf_exo)-1]
                #matrix is important for the exogenous forecast
                matrix[vector_zeros[time-1],0]=result_y1
            #the rest is used for the endogenous forecast
            result_y=predictions #the actual endogenous prediction
            #endogenous accuracy measures:
            dates_y=test_dates
            rms_y=rms_endo
            rms_yy=rms
            mae_y=mae_endo
            mape_y=mape
            acc_y=accuracy
            mse_y=mse 
            r2_y=r2
            r2score_y=r2score
            r2_endo_y=r2_endo
        elif scenario==2:
            if r2_future>0.8:
                if model=='xgb':
                    result_c1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_c1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],1]=result_c1
            result_c=predictions
            dates_c=test_dates
            rms_c=rms_endo
            mae_c=mae 
            mape_c=mape
            acc_c=accuracy
            r2_c=r2 
            mse_c=mse 
            r2score_c=r2score
            r2_endo_c=r2_endo
        elif scenario==3:
            if r2_future>0.8:
                if model=='xgb':
                    result_yea1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_yea1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],2]=result_yea1
            result_yea=predictions
            dates_yea=test_dates
            rms_yea=rms_endo
            mae_yea=mae 
            mape_yea=mape
            acc_yea=accuracy
            r2_yea=r2 
            mse_yea=mse 
            r2score_yea=r2score
            r2_endo_yea=r2_endo
        elif scenario==4:
            if r2_future>0.7:
                if model=='xgb':
                    result_x1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_x1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],3]=result_x1
            result_x=predictions
            dates_x=test_dates
            rms_x=rms_endo
            mae_x=mae 
            mape_x=mape
            acc_x=accuracy
            r2_x=r2 
            mse_x=mse 
            r2score_x=r2score
            r2_endo_x=r2_endo
        elif scenario==5:
            if r2_future>0.7:
                if model=='xgb':
                    result_m1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_m1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],4]=result_m1
            result_m=predictions
            dates_m=test_dates
            rms_m=rms_endo
            mae_m=mae 
            mape_m=mape
            acc_m=accuracy
            r2_m=r2 
            mse_m=mse 
            r2score_m=r2score
            r2_endo_m=r2_endo
        elif scenario==6:
            if r2_future>0.8:
                if model=='xgb':
                    result_r1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_r1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],5]=result_r1
            result_R=predictions
            dates_R=test_dates
            rms_r=rms_endo
            mae_r=mae 
            mape_r=mape
            acc_r=accuracy
            r2_r=r2 
            mse_r=mse 
            r2score_r=r2score
            r2_endo_r=r2_endo
        elif scenario==7:
            if r2_future>0.8:
                if model=='xgb':
                    result_con1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_con1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],6]=result_con1
            result_Con=predictions
            dates_Con=test_dates
            rms_con=rms_endo
            mae_con=mae 
            mape_con=mape
            acc_con=accuracy
            r2_con=r2 
            mse_con=mse 
            r2score_con=r2score
            r2_endo_con=r2_endo
        elif scenario==8:
            if r2_future>0.8:
                if model=='xgb':
                    result_pmi1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_pmi1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],7]=result_pmi1
            result_pmi=predictions
            dates_pmi=test_dates
            rms_pmi=rms_endo
            mae_pmi=mae 
            mape_pmi=mape
            acc_pmi=accuracy
            r2_pmi=r2 
            mse_pmi=mse 
            r2score_pmi=r2score
            r2_endo_pmi=r2_endo
        elif scenario==9:
            if r2_future>0.8:
                if model=='xgb':
                    result_ind1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_ind1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],8]=result_ind1
            result_Ind=predictions
            dates_Ind=test_dates
            rms_ind=rms_endo
            mae_ind=mae 
            mape_ind=mape
            acc_ind=accuracy
            r2_ind=r2 
            mse_ind=mse 
            r2score_ind=r2score
            r2_endo_ind=r2_endo
        elif scenario==10:
            if r2_future>0.8:
                if model=='xgb':
                    result_const1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_const1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],9]=result_const1
            result_Const=predictions
            dates_Const=test_dates
            rms_const=rms_endo
            mae_const=mae 
            mape_const=mape
            acc_const=accuracy
            r2_const=r2 
            mse_const=mse 
            r2score_const=r2score
            r2_endo_const=r2_endo
        elif scenario==11:
            if r2_future>0.6:
                if model=='xgb':
                    result_w1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_w1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],10]=result_w1
            result_w=predictions
            dates_w=test_dates
            rms_w=rms_endo
            mae_w=mae 
            mape_w=mape
            acc_w=accuracy
            r2_w=r2 
            mse_w=mse 
            r2score_w=r2score
            r2_endo_w=r2_endo
        elif scenario==12:
            if r2_future<1:
                if model=='xgb':
                    result_nx1=prediction_xgb_exo[len(prediction_xgb_exo)-1]
                if model=='rf':
                    result_nx1=prediction_rf_exo[len(prediction_rf_exo)-1]
                matrix[vector_zeros[time-1],11]=result_nx1
            result_nx=predictions
            dates_nx=test_dates
            rms_nx=rms_endo
            mae_nx=mae 
            mape_nx=mape
            acc_nx=accuracy
            r2_nx=r2 
            mse_nx=mse 
            r2score_nx=r2score
            r2_endo_nx=r2_endo

            test_dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
            test_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in test_dates]
            
            #append the endogenous prediction values to the original dataset1
            for pred in range(1,15):
                if r2_endo_y[pred]>0.93 and rms_y[pred]<0.9: #the endogenous accuracy measure
                    date_y=dates_y[pred]
                    dataset1=dataset1.append({'day' : date_y.day, 'month' : date_y.month, 'year' : date_y.year, 'dy' : result_y[pred] , 'rms_y' : rms_y[pred], 'mae_y' : mae_y[pred], 'acc_y' : r2_endo_y[pred], 'r2_y' : r2_future, 'mse' : mse_y, 'r2score' : r2score_y}, ignore_index=True) #
            
            for predc in range(1,15):
                if r2_endo_c[predc]>0.85 and rms_c[predc]<0.8:
                    date_c=dates_c[predc]
                    dataset1=dataset1.append({'day' : date_c.day, 'month' : date_c.month, 'year' : date_c.year, 'dc' : result_c[predc], 'rms_c' : rms_c[predc], 'mae_c' : mae_c, 'acc_c' : r2_endo_c[predc], 'r2_c' : r2_future, 'mse' : mse_c, 'r2score' : r2score_c} , ignore_index=True)
            
            for predyea in range(1,15): 
                if r2_endo_yea[predyea]>0.9 and rms_yea[predyea]<0.8: 
                    date_yea=dates_yea[predyea]
                    dataset1=dataset1.append({'day' : date_yea.day, 'month' : date_yea.month, 'year' : date_yea.year, 'dyea' : result_yea[predyea], 'rms_yea' : rms_yea[predyea], 'mae_yea' : mae_yea, 'acc_yea' : r2_endo_yea[predyea], 'r2_yea' : r2_future, 'mse' : mse_yea, 'r2score' : r2score_yea} , ignore_index=True)
            
            for predx in range(1,15): 
                if r2_endo_x[predx]>0.85 and rms_x[predx]<4:   
                    date_x=dates_x[predx]
                    dataset1=dataset1.append({'day' : date_x.day, 'month' : date_x.month, 'year' : date_x.year, 'dx' : result_x[predx], 'rms_x' : rms_x[predx], 'mae_x' : mae_x, 'acc_x' : acc_x, 'r2_x' : r2_future, 'mse' : mse_x, 'r2score' : r2score_x} , ignore_index=True)

            for predm in range(1,15): 
                if r2_endo_m[predm]>0.75 and rms_m[predm]<4: 
                    date_m=dates_m[predm]
                    dataset1=dataset1.append({'day' : date_m.day, 'month' : date_m.month, 'year' : date_m.year, 'dm' : result_m[predm] , 'rms_m' : rms_m[predm]}, ignore_index=True)
      
            for predr in range(1,15):  
                if r2_endo_r[predr]>0.9 and rms_r[predr]<0.2:  
                    date_r=dates_R[predr]
                    dataset1=dataset1.append({'day' : date_r.day, 'month' : date_r.month, 'year' : date_r.year, 'dR' : result_R[predr], 'rms_r' : rms_r[predr], 'mae_r' : mae_r, 'acc_r' : acc_r, 'r2_r' : r2_future, 'mse' : mse_r, 'r2score' : r2score_r} , ignore_index=True)
            
            for predcon in range(1,15): 
                if r2_endo_con[predcon]>0.85 and rms_con[predcon]<2: 
                    date_con=dates_Con[predcon]
                    dataset1=dataset1.append({'day' : date_con.day, 'month' : date_con.month, 'year' : date_con.year, 'dConSen_bal' : result_Con[predcon], 'rms_con' : rms_con[predcon], 'mae_con' : mae_con, 'acc_con' : acc_con, 'r2_con' : r2_future, 'mse' : mse_con, 'r2score' : r2score_con} , ignore_index=True)
            
            for predpmi in range(1,15):  
                if r2_endo_pmi[predpmi]>0.85 and rms_pmi[predpmi]<1.5:
                    date_pmi=dates_pmi[predpmi]
                    dataset1=dataset1.append({'day' : date_pmi.day, 'month' : date_pmi.month, 'year' : date_pmi.year, 'dPMI_manuf' : result_pmi[predpmi], 'rms_pmi' : rms_pmi[predpmi], 'mae_pmi' : mae_pmi, 'acc_pmi' : acc_pmi, 'r2_pmi' : r2_future, 'mse' : mse_pmi, 'r2score' : r2score_pmi} , ignore_index=True)
            
            for predind in range(1,15): 
                if r2_endo_ind[predind]>0.85 and rms_ind[predind]<3:   
                    date_ind=dates_Ind[predind]
                    dataset1=dataset1.append({'day' : date_ind.day, 'month' : date_ind.month, 'year' : date_ind.year, 'dInd_conf' : result_Ind[predind], 'rms_ind' : rms_ind[predind], 'mae_ind' : mae_ind, 'acc_ind' : acc_ind, 'r2_ind' : r2_future, 'mse' : mse_ind, 'r2score' : r2score_ind} , ignore_index=True)
            
            for predconst in range(1,15): 
                if r2_endo_const[predconst]>0.85 and rms_const[predconst]<4:   
                    date_const=dates_Const[predconst]
                    dataset1=dataset1.append({'day' : date_const.day, 'month' : date_const.month, 'year' : date_const.year, 'dConst_conf' : result_Const[predconst], 'rms_const' : rms_const[predconst], 'mae_const' : mae_const, 'acc_const' : acc_const, 'r2_const' : r2_future, 'mse' : mse_const, 'r2score' : r2score_const} , ignore_index=True)
            
            for predw in range(1,15):  
                if r2_endo_w[predw]>0.65 and rms_w[predw]<1.5:   
                    date_w=dates_w[predw]
                    dataset1=dataset1.append({'day' : date_w.day, 'month' : date_w.month, 'year' : date_w.year, 'dw' : result_w[predw], 'rms_w' : rms_w[predw], 'mae_w' : mae_w, 'acc_w' : acc_w, 'r2_w' : r2_future, 'mse' : mse_w, 'r2score' : r2score_w} , ignore_index=True)
 
            for prednx in range(1,15):  
                if r2_endo_nx[prednx]<1:  
                    date_nx=dates_nx[prednx]
                    dataset1=dataset1.append({'day' : date_nx.day, 'month' : date_nx.month, 'year' : date_nx.year, 'dnx' : result_nx[prednx]}, ignore_index=True)

            #convert the dataset1 into csv file for the endogenous forecast
            if model=='xgb':
                dataset1.to_csv('GDP_tree_VARYoYry_Yxgb03.csv', index=False)
            if model=='rf':
                dataset1.to_csv('GDP_tree_VARYoYry_Yrf02.csv', index=False)
print('matrix', matrix)
print('vector', vector_zeros)

## GDP GRAPH

In [21]:
model='xgb'

import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import plotly
import chart_studio
chart_studio.tools.set_credentials_file(username='sgawthor', api_key='wa3B5oJOSFTqcBuopZI2')
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
import numpy as geek

#First run the model only for the first scenario - the GDP model
for x in range(1,2):
    scenario=x
    if model=='xgb':
        dataset = pd.read_csv('GDP_tree_VARYoY1_Yexo1_xgb.csv') #GDP_tree_VARlags_select.csv
    if model=='rf':
        dataset = pd.read_csv('GDP_tree_VARYoY1_Yexo1_rf.csv')
    #1=y
    hh=dataset.head()
    X = dataset.iloc[0:71, 0:14].values
    y = dataset.iloc[0:71, 14].values

    #2=c
    yc=dataset.iloc[0:71, 15].values
    Xc=dataset.iloc[0:71, 0:14].values

    #3=yea
    yea=dataset.iloc[0:71, 16].values
    Xea=dataset.iloc[0:71, 0:14].values

    #4=x
    yx=dataset.iloc[0:71, 17].values
    Xx=dataset.iloc[0:71, 0:14].values

    #5=m
    ym=dataset.iloc[0:71, 18].values
    Xm=dataset.iloc[0:71, 0:14].values

    #6=R
    yR=dataset.iloc[0:71, 19].values
    XR=dataset.iloc[0:71, 0:14].values

    #8=ConSen_bal
    yCon=dataset.iloc[0:71, 20].values
    XCon=dataset.iloc[0:71, 0:14].values

    #10=PMI_manuf
    yPMI_manuf=dataset.iloc[0:71, 21].values
    XPMI_manuf=dataset.iloc[0:71, 0:14].values

    #11=Ind_conf
    yInd_conf=dataset.iloc[0:71, 22].values
    XInd_conf=dataset.iloc[0:71, 0:14].values

    #12=Const_conf
    yConst_conf=dataset.iloc[0:71, 23].values
    XConst_conf=dataset.iloc[0:71, 0:14].values

    #13=w
    yw=dataset.iloc[0:71, 24].values
    Xw=dataset.iloc[0:71, 0:14].values
    

    if scenario==1:
        name='dy'
        features=X
        labels=y
    elif scenario==2:
        name='dc'
        features=Xc
        labels=yc
    elif scenario==3:
        name='dyea'
        features=Xea
        labels=yea
    elif scenario==4:
        name='dx'
        features=Xx
        labels=yx
    elif scenario==5:
        name='dm'
        features=Xm
        labels=ym
    elif scenario==6:
        name='dR'
        features=XR
        labels=yR
    elif scenario==7:
        name='dConSen_bal'
        features=XCon
        labels=yCon
    elif scenario==8:
        name='dPMI_manuf'
        features=XPMI_manuf
        labels=yPMI_manuf
    elif scenario==9:
        name='dInd_conf'
        features=XInd_conf
        labels=yInd_conf
    elif scenario==10:
        name='dConst_conf'
        features=XConst_conf
        labels=yConst_conf
    elif scenario==11:
        name='dw'
        features=Xw
        labels=yw    

    from sklearn.model_selection import train_test_split

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
    #print('Training Features Shape:', train_features.shape)
    #print('Training Labels Shape:', train_labels.shape)
    #print('Testing Features Shape:', test_features.shape)
    #print('Testing Labels Shape:', test_labels.shape)

 
    # Saving feature names for later use
    feature_list=['day', 'month', 'year', 'Ldy', 'dc', 'dyea', 'dx',	'dm', 'dR', 'dConSen_bal', 'dPMI_manuf', 'dInd_conf', 'dConst_conf', 'dw']
    baseline_preds = test_features[:, feature_list.index('dyea')]
    #print(baseline_preds)

    # Baseline errors, and display average baseline error
    baseline_errors = abs(baseline_preds - test_labels)
    #print('Average baseline error: ', round(np.mean(baseline_errors), 2))

    # Import the model we are using
    from sklearn.ensemble import RandomForestRegressor
    # Instantiate model with 1000 decision trees
    rf = RandomForestRegressor(n_estimators= 1000, random_state=42)
    xgb = XGBRegressor()

    # Train the model on training data
    rf.fit(train_features, train_labels)
    xgb.fit(train_features, train_labels)
    #print('train_labels', train_labels)

    # Use the forest's predict method on the test data
    predictions_xgb=xgb.predict(test_features)
    predictions_rf=rf.predict(test_features)
    if model=='rf':
        predictions=predictions_rf
    if model=='xgb':
        predictions=predictions_xgb

    # Calculate the absolute errors
    errors = abs(predictions - test_labels)
    # Print out the mean absolute error (mae)
    #print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.') #the average distance btw predicted and actual value

    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * abs(errors /(test_labels))
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    #print('Accuracy:', round(accuracy, 2), '%.')
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    from sklearn.metrics import r2_score

    rms = sqrt(mean_squared_error(test_labels, predictions)) 
    #print('RMSE', rms)

    # from sklearn.metrics import mean_squared_error, r2_score
    model_score =  metrics.r2_score(test_labels, predictions)
    #print('coefficient of determination R^2 of the prediction.: ',model_score)

    #print("Mean squared error: %.2f"% mean_squared_error(test_labels, predictions))
    # Explained variance score: 1 is perfect prediction
    #print('Test Variance score: %.2f' % r2_score(test_labels, predictions))

    score=rf.score(test_features, test_labels)
    #print('score', score)
    #print('pred', predictions)
    #print(test_labels)

    # #--------------------Variable importance
    # Get numerical feature importances
    importances = list(rf.feature_importances_)
    # List of tuples with variable and importance
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
    # Sort the feature importances by most important first
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    # Print out the feature and importances 
    #[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

 #------------------VISUALIZATION

 #----------- First visualize the variables' importance in the model
    import matplotlib.pyplot as plt

    plt.style.use('fivethirtyeight')
    fig1, ax = plt.subplots(facecolor=(1,1,1))
    x_values = list(range(len(importances)))
    # Make a bar chart
    plt.bar(x_values, importances, orientation = 'vertical')
    ax = plt.gca()
    ax.set_facecolor('linen') 
    # Tick labels for x axis
    feature_list1=['day', 'month', 'year', 'y', 'c', 'yea', 'x',	'm', 'R', 'HHs conf.', 'PMI manuf.', 'Ind conf.', 'Const conf.', 'w']
    plt.xticks(x_values, feature_list1, rotation='vertical')
    plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances')
    fig1 = plt.gcf()
    ax.set_facecolor('white')
    #plt.show()

    #Write graph to a png file
    var_name = 'Forest_importance_Y'
    var_name += name
    var_name += '.png'
    fig1.savefig(var_name, bbox_inches='tight', dpi=100, facecolor=fig1.get_facecolor())# edgecolor='none'

 #----------- Second visualize the predictions from the previously estimated models
 # read the csv files with the previously estimated results
    dataset1 = pd.read_csv('GDP_tree_VARYoYry_Yrf_final.csv') #rf
    dataset2 = pd.read_csv('GDP_tree_VARYoY1_Yexo1_rf_final.csv') #rf

    dataset3 = pd.read_csv('GDP_tree_VARYoYry_Yxgb_final.csv') #xgb
    dataset4 = pd.read_csv('GDP_tree_VARYoY1_Yexo1_xgb_final.csv') #xgb

    dataset_names=['day',	'month',	'year',	'dy',	'dc',	'dyea',	'dx',	'dm',	'dR',	'dConSen_bal',	'dPMI_manuf',	'dInd_conf',	'dConst_conf',	'dw', 'yt',	'ct',	'yeat',	'xt',	'mt',	'Rt',	'ConSen_balt',	'PMI_manuft',	'Ind_conft',	'Const_conft',	'wt']

    #get all the values into a dataframe
    dataset_pd=dataset2.iloc[0:79, 0:25].values 
    dataset_pd1=dataset1.iloc[72:7766, 0:25].values
    dataset_pd_xgb=dataset4.iloc[0:79, 0:25].values
    dataset_pd1_xgb=dataset3.iloc[72:7766, 0:25].values 
    dfObj = pd.DataFrame(dataset_pd, columns=dataset_names)
    dfObj1=pd.DataFrame(dataset_pd1,columns=dataset_names)
    dfObj=dfObj.append(dfObj1)
    dfObj_xgb = pd.DataFrame(dataset_pd_xgb, columns=dataset_names)
    dfObj1_xgb=pd.DataFrame(dataset_pd1_xgb,columns=dataset_names)
    dfObj_xgb=dfObj_xgb.append(dfObj1_xgb)
    
    #turn these dataframes into csv files
    dfObj_xgb.to_csv('GDP_xgb_blog.csv', index=False)
    dataset4 = pd.read_csv('GDP_xgb_blog.csv')

    dfObj.to_csv('GDP_rf_blog.csv', index=False)
    dataset2 = pd.read_csv('GDP_rf_blog.csv') 

    #read Czech national bank and Ministry of finance predictions
    dataset_mfcnb = pd.read_csv('cnb_mf_predikce.csv')

    #GRAPH the datasets
    import matplotlib.pyplot as plt
    # Use datetime for creating date objects for plotting
    import datetime
    # Dates of training values
    months = dataset2.iloc[0:78, 1].values
    days = dataset2.iloc[0:78, 0].values
    years = dataset2.iloc[0:78, 2].values
    months_xgb = dataset4.iloc[0:78, 1].values
    days_xgb = dataset4.iloc[0:78, 0].values
    years_xgb = dataset4.iloc[0:78, 2].values
    months_mfcnb = dataset_mfcnb.iloc[0:54, 1].values
    days_mfcnb = dataset_mfcnb.iloc[0:54, 0].values
    years_mfcnb = dataset_mfcnb.iloc[0:54, 2].values
    months_mf = dataset_mfcnb.iloc[0:54, 1].values
    days_mf = dataset_mfcnb.iloc[0:54, 0].values
    years_mf = dataset_mfcnb.iloc[0:54, 2].values

    # List and then convert to datetime object
    dates_endo = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
    dates_endo = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in dates_endo]

    dates_endo_xgb = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_xgb, months_xgb, days_xgb)]
    dates_endo_xgb = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in dates_endo_xgb]

    dates_endo_mfcnb = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_mfcnb, months_mfcnb, days_mfcnb)]
    dates_endo_mfcnb = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in dates_endo_mfcnb]

    # Dataframe with true values and dates
    true_data_endo = pd.DataFrame(data = {'date_endo': dates_endo[0:71], 'actual_endo': dataset2.iloc[0:71, 2+x].values})
    # Dataframe with future predicted values
    predictions_data = pd.DataFrame(data = {'date_endo': dates_endo[70:78], 'prediction': dataset2.iloc[70:78, 2+x].values})
    predictions_data_xgb = pd.DataFrame(data = {'date_endo': dates_endo_xgb[70:78], 'prediction': dataset4.iloc[70:78, 2+x].values})
    # Dates of predictions
    months = dataset2.iloc[79:7766, 1].values #1134
    days = dataset2.iloc[79:7766, 0].values
    years = dataset2.iloc[79:7766, 2].values
    months_xgb = dataset4.iloc[79:7766, 1].values
    days_xgb = dataset4.iloc[79:7766, 0].values
    years_xgb = dataset4.iloc[79:7766, 2].values
    months_cnb = dataset_mfcnb.iloc[0:54, 1].values
    days_cnb = dataset_mfcnb.iloc[0:54, 0].values
    years_cnb = dataset_mfcnb.iloc[0:54, 2].values
    months_mf = dataset_mfcnb.iloc[0:54, 1].values
    days_mf = dataset_mfcnb.iloc[0:54, 0].values
    years_mf = dataset_mfcnb.iloc[0:54, 2].values
    # Column of dates
    test_dates_endo = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
    test_dates_endo_xgb = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_xgb, months_xgb, days_xgb)]
    test_dates_endo_cnb = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_cnb, months_cnb, days_cnb)]
    test_dates_endo_mf = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_mf, months_mf, days_mf)]
    # Convert to datetime objects
    test_dates_endo = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in test_dates_endo]
    test_dates_endo_xgb = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in test_dates_endo_xgb]
    test_dates_endo_cnb = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in test_dates_endo_cnb]
    test_dates_endo_mf = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in test_dates_endo_mf]
    # Dataframe with predictions and dates
    predictions_data_endo = pd.DataFrame(data = {'date_endo': test_dates_endo, 'prediction_endo': dataset2.iloc[79:7766, 2+x].values})
    predictions_data_endo_xgb = pd.DataFrame(data = {'date_endo': test_dates_endo_xgb, 'prediction_endo': dataset4.iloc[79:7766, 2+x].values})
    predictions_data_endo_cnb = pd.DataFrame(data = {'date_endo': test_dates_endo_cnb, 'prediction_endo': dataset_mfcnb.iloc[0:54, 3].values})
    predictions_data_endo_mf = pd.DataFrame(data = {'date_endo': test_dates_endo_mf, 'prediction_endo': dataset_mfcnb.iloc[0:54, 4].values})

    #Names of variables
    variab = ['Gross domestic product', 'Consumption', 'GDP EA', 'Export', 'IMPORT', 'PRIBOR', 'Consumption sentiment', 'PMI manufacturing', 'Construction confidence', 'Industrial confidence', 'Wage']

    ## EVIEWS results
    dataset_ev = pd.read_csv('eviews_forecast.csv') 
    dataset_ev=dataset_ev.iloc[46:89, 0:27].values
    dataset_evnames=['day', 'month', 'year', 'gdp_vecm', 'gdp_fabvarx', 'gdp_fabvar', 'gdp_forecast', 'cons_fabvar', 'cons_favar', 'cons_var', 'cons_forecast', 'im_bvarx', 'im_fabvarx', 'im_forecast', 'infl_bvarx', 'infl_forecast', 'wage_fabvar', 'wage_favar', 'wage_forecast', 'inv_fabvar', 'inv_var', 'inv_forecast', 'lab_vecm', 'lab_forecast', 'ex_bvarx', 'ex_fabvarx', 'ex_forecast']
    dfObjev = pd.DataFrame(dataset_ev, columns=dataset_evnames)
    dfObjev.replace('#N/A', 4)
    dfObjev.to_csv('GDP_eviews.csv', index=False)
    dataset_eviews = pd.read_csv('GDP_eviews.csv')

    import matplotlib.pyplot as plt
    import datetime

    #create dates for the eviews file
    months = dataset_eviews.iloc[0:38, 1].values #43
    days = dataset_eviews.iloc[0:38, 0].values
    years = dataset_eviews.iloc[0:38, 2].values
    months_exo = dataset_eviews.iloc[37:43, 1].values #43
    days_exo = dataset_eviews.iloc[37:43, 0].values
    years_exo = dataset_eviews.iloc[37:43, 2].values

    # Convert to datetime objects
    dates_eviews = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
    dates_eviews = [datetime.datetime.strptime(date_eviews, '%Y-%m-%d') for date_eviews in dates_eviews]
    dates_eviews_exo = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_exo, months_exo, days_exo)]
    dates_eviews_exo = [datetime.datetime.strptime(date_eviews, '%Y-%m-%d') for date_eviews in dates_eviews_exo]
    eviews_gdp_forecast= pd.DataFrame(data = {'date_eviews': dates_eviews, 'prediction': dataset_eviews.iloc[0:38, 6].values})
    eviews_gdp_forecast_exo= pd.DataFrame(data = {'date_eviews': dates_eviews_exo, 'prediction': dataset_eviews.iloc[37:43, 6].values})

    # average forecast from the RF, XGB models as one forecast value and from the Eviews as another one
    future_gdp=dataset2.iloc[70:76,3].values
    future_gdp[1:6]=((dataset2.iloc[71:76,3].values+dataset4.iloc[71:76,3].values)/2+dataset_eviews.iloc[38:43, 6].values)/2

    # plot the results
    overlapping = 0.150
    fig = px.line(true_data_endo, x='date_endo', y='actual_endo', labels={"actual_endo":"", "date_endo":"years"}, title=variab[x-1])
    fig.update_layout(title={'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'})
    fig.update_traces(line_color='black')

    fig.add_traces(go.Scatter(name="XGB endo",x=predictions_data_endo_xgb['date_endo'], y=predictions_data_endo_xgb['prediction_endo'], mode = 'markers', marker=dict(color='rgba(128,0,128, 0.8)', size=7,line=dict(color='darkblue', width=1))))
    fig.add_trace(go.Scatter(name="VAR_gdp",x=eviews_gdp_forecast['date_eviews'], y= eviews_gdp_forecast['prediction'], mode = 'markers', marker=dict(color='rgba(105,105,105, 0.8)', size=7, line=dict(color='darkblue', width=1))), row=x,col=1)
    fig.add_traces(go.Scatter(name="RF endo",x=predictions_data_endo['date_endo'], y=predictions_data_endo['prediction_endo'], mode = 'markers', marker=dict(color='rgba(255,192,203, 0.8)',size=7, line=dict(color='darkblue', width=1))))

    if scenario==1: #the GDP model
        ex_forecast= pd.DataFrame(data = {'date_endo': dates_endo[70:76], 'prediction': future_gdp})
        fig.add_trace(go.Scatter(x=ex_forecast['date_endo'], y=ex_forecast['prediction'], mode = 'lines', marker=dict(color='crimson'), showlegend=False), row=x,col=1) 
        fig.add_traces(go.Scatter(name="cnb",x=predictions_data_endo_cnb['date_endo'], y=predictions_data_endo_cnb['prediction_endo'], mode = 'markers', marker=dict(color='rgba(0,0,205, 0.8)', size=7, line=dict(color='darkblue', width=1))))

        fig.add_traces(go.Scatter(name="mf",x=predictions_data_endo_mf['date_endo'], y=predictions_data_endo_mf['prediction_endo'], mode = 'markers', marker=dict(color='rgba(0,255,255, 0.8)', size=7, line=dict(color='darkblue', width=1))))
        

    fig.update_layout(legend=dict(orientation="h",
    yanchor="bottom",
    y=-0.8,
    xanchor="right",
    x=1
    ))

    fig.update_xaxes(rangeslider_visible=True)

    fig.show()

    #if you want it to appear in plotly:
    #py.plot(fig, filename='random_forest2_new', auto_open=True)






In [17]:
## GRAPHS FOR THE REMAINING VARIABLES

In [23]:

model='xgb' #type 'rf' if you want the variable importance graphs for the random forest model

import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import plotly
import chart_studio
chart_studio.tools.set_credentials_file(username='sgawthor', api_key='wa3B5oJOSFTqcBuopZI2')
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
import numpy as geek
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#set subplots
fig = make_subplots(rows=4, cols=1, subplot_titles=("GDP", "Consumption", "Export", "Import","Interest rate"), vertical_spacing=0.05)

for x in range(1,5): #get results for the first four variables
    scenario=x
    if model=='xgb':
        dataset = pd.read_csv('GDP_tree_VARYoY1_Yexo1_xgb.csv')
    if model=='rf':
        dataset = pd.read_csv('GDP_tree_VARYoY1_Yexo1_rf.csv')
    #1=y
    hh=dataset.head()
    X = dataset.iloc[0:71, 0:14].values #25
    y = dataset.iloc[0:71, 14].values

    #2=c
    yc=dataset.iloc[0:71, 15].values
    Xc=dataset.iloc[0:71, 0:14].values

    #3=yea
    yea=dataset.iloc[0:71, 16].values
    Xea=dataset.iloc[0:71, 0:14].values

    #4=x
    yx=dataset.iloc[0:71, 17].values
    Xx=dataset.iloc[0:71, 0:14].values

    #5=m
    ym=dataset.iloc[0:71, 18].values
    Xm=dataset.iloc[0:71, 0:14].values

    #6=R
    yR=dataset.iloc[0:71, 19].values
    XR=dataset.iloc[0:71, 0:14].values

    #7=ConSen_bal
    yCon=dataset.iloc[0:71, 20].values
    XCon=dataset.iloc[0:71, 0:14].values

    #8=PMI_manuf
    yPMI_manuf=dataset.iloc[0:71, 21].values
    XPMI_manuf=dataset.iloc[0:71, 0:14].values

    #9=Ind_conf
    yInd_conf=dataset.iloc[0:71, 22].values
    XInd_conf=dataset.iloc[0:71, 0:14].values

    #10=Const_conf
    yConst_conf=dataset.iloc[0:71, 23].values
    XConst_conf=dataset.iloc[0:71, 0:14].values

    #11=w
    yw=dataset.iloc[0:71, 24].values
    Xw=dataset.iloc[0:71, 0:14].values
    

    if scenario==1:
        name='dy'
        features=X
        labels=y
    elif scenario==2:
        name='dc'
        features=Xc
        labels=yc
    elif scenario==3:
        name='dyea'
        features=Xea
        labels=yea
    elif scenario==4:
        name='dx'
        features=Xx
        labels=yx
    elif scenario==5:
        name='dm'
        features=Xm
        labels=ym
    elif scenario==6:
        name='dR'
        features=XR
        labels=yR
    elif scenario==7:
        name='dConSen_bal'
        features=XCon
        labels=yCon
    elif scenario==8:
        name='dPMI_manuf'
        features=XPMI_manuf
        labels=yPMI_manuf
    elif scenario==9:
        name='dInd_conf'
        features=XInd_conf
        labels=yInd_conf
    elif scenario==10:
        name='dConst_conf'
        features=XConst_conf
        labels=yConst_conf
    elif scenario==11:
        name='dw'
        features=Xw
        labels=yw    

    from sklearn.model_selection import train_test_split

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
    #print('Training Features Shape:', train_features.shape)
    #print('Training Labels Shape:', train_labels.shape)
    #print('Testing Features Shape:', test_features.shape)
    #print('Testing Labels Shape:', test_labels.shape)

 
    # Saving feature names for later use
    feature_list=['day', 'month', 'year', 'Ldy', 'dc', 'dyea', 'dx',	'dm', 'dR', 'dConSen_bal', 'dPMI_manuf', 'dInd_conf', 'dConst_conf', 'dw']
    baseline_preds = test_features[:, feature_list.index('dyea')]
    #print(baseline_preds)

    # Baseline errors, and display average baseline error
    baseline_errors = abs(baseline_preds - test_labels)
    #print('Average baseline error: ', round(np.mean(baseline_errors), 2))

    # Import the model we are using
    from sklearn.ensemble import RandomForestRegressor
    # Instantiate model with 1000 decision trees
    rf = RandomForestRegressor(n_estimators= 1000, random_state=42)
    xgb = XGBRegressor()

    # Train the model on training data
    rf.fit(train_features, train_labels)
    xgb.fit(train_features, train_labels)
    #print('train_labels', train_labels)

    # Use the forest's predict method on the test data
    predictions_xgb=xgb.predict(test_features)
    predictions_rf=rf.predict(test_features)
    if model=='rf':
        predictions=predictions_rf
    if model=='xgb':
        predictions=predictions_xgb

    # Calculate the absolute errors
    errors = abs(predictions - test_labels)
    # Print out the mean absolute error (mae)
    #print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.') #the average distance btw predicted and actual value
   
    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * abs(errors /(test_labels))
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape) #one outlier prediction can bias it
    #print('Accuracy:', round(accuracy, 2), '%.')
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    from sklearn.metrics import r2_score

    rms = sqrt(mean_squared_error(test_labels, predictions)) #rms = sqrt(mean_squared_error(y_actual, y_predicted))
    #print('RMSE', rms)

    # from sklearn.metrics import mean_squared_error, r2_score
    model_score =  metrics.r2_score(test_labels, predictions)
    #print('coefficient of determination R^2 of the prediction.: ',model_score)

    #print("Mean squared error: %.2f"% mean_squared_error(test_labels, predictions))
    # Explained variance score: 1 is perfect prediction
    #print('Test Variance score: %.2f' % r2_score(test_labels, predictions))

    score=rf.score(test_features, test_labels)
    #print('score', score)
    #print('pred', predictions)
    #print(test_labels)

    # #--------------------Variable importance
    # Get numerical feature importances
    importances = list(rf.feature_importances_)
    # List of tuples with variable and importance
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
    # Sort the feature importances by most important first
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    # Print out the feature and importances 
    #[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

 #------------------VISUALIZATION
 # FIRST VISUALIZE THE VARIABLES' IMPORTANCE FOR EACH MODEL

#   bar plot of the feature importances 
    import matplotlib.pyplot as plt

    #matplotlib inline
    # Set the style
    plt.style.use('fivethirtyeight')
    fig1, ax = plt.subplots(facecolor=(1,1,1)) 
     # list of x locations for plotting
    x_values = list(range(len(importances)))
    # Make a bar chart
    plt.bar(x_values, importances, orientation = 'vertical')
    ax = plt.gca()
    ax.set_facecolor('linen')
    # Tick labels for x axis
    feature_list1=['day', 'month', 'year', 'y', 'c', 'yea', 'x',	'm', 'R', 'HHs conf.', 'PMI manuf.', 'Ind conf.', 'Const conf.', 'w']
    plt.xticks(x_values, feature_list1, rotation='vertical')
    # Axis labels and title
    plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances')
    fig1 = plt.gcf()
    ax.set_facecolor('white')
    #plt.show()

    # Write graph to a png file
    var_name = 'Forest_importance_Y'
    var_name += name
    var_name += '.png'
    fig1.savefig(var_name, bbox_inches='tight', dpi=100, facecolor=fig1.get_facecolor())# edgecolor='none'

 # SECOND VISUALIZE THE PREDICTIONS FROM THE PREVIOUSLY ESTIMATED MODELS
 # Read the csv files with the obtained values
    dataset1 = pd.read_csv('GDP_tree_VARYoYry_Yrf_final.csv') #rf
    dataset2 = pd.read_csv('GDP_tree_VARYoY1_Yexo1_rf_final.csv') #rf

    dataset3 = pd.read_csv('GDP_tree_VARYoYry_Yxgb_final.csv') #xgb
    dataset4 = pd.read_csv('GDP_tree_VARYoY1_Yexo1_xgb_final.csv') #xgb

    dataset_names=['day',	'month',	'year',	'dy',	'dc',	'dx',	'dm',	'dR',	'dConSen_bal',	'dPMI_manuf',	'dInd_conf',	'dConst_conf',	'dw', 'yt',	'ct',	'yeat',	'xt',	'mt',	'Rt',	'ConSen_balt',	'PMI_manuft',	'Ind_conft',	'Const_conft',	'wt', 'dyea']
    dataset_pd=dataset2.iloc[0:79, 0:25].values #55
    dataset_pd[0:79,5:24]=dataset2.iloc[0:79, 6:25].values
    dataset_pd1=dataset1.iloc[72:4000, 0:25].values
    dataset_pd1[:,5:24]=dataset1.iloc[72:4000, 6:25].values
    dataset_pd_xgb=dataset4.iloc[0:79, 0:25].values #55
    dataset_pd_xgb[:,5:24]=dataset4.iloc[0:79, 6:25].values
    dataset_pd1_xgb=dataset3.iloc[72:4000, 0:25].values
    dataset_pd1_xgb[:,5:24]=dataset3.iloc[72:4000, 6:25].values

    #create dataframes
    dfObj = pd.DataFrame(dataset_pd, columns=dataset_names)
    dfObj1=pd.DataFrame(dataset_pd1,columns=dataset_names)
    dfObj=dfObj.append(dfObj1) #append the endogenous forecasts to the datafile containing also exogenous forecasts
    dfObj_xgb = pd.DataFrame(dataset_pd_xgb, columns=dataset_names)
    dfObj1_xgb=pd.DataFrame(dataset_pd1_xgb,columns=dataset_names)
    dfObj_xgb=dfObj_xgb.append(dfObj1_xgb) #append the endogenous forecasts to the datafile containing also exogenous forecasts

    dfObj_xgb.to_csv('GDP_xgb_blog.csv', index=False)
    dataset4 = pd.read_csv('GDP_xgb_blog.csv')

    dfObj.to_csv('GDP_rf_blog.csv', index=False)
    dataset2 = pd.read_csv('GDP_rf_blog.csv') 

    #reat csv with the Ministry of Finance and the Czech National Bank predictions
    dataset_mfcnb = pd.read_csv('cnb_mf_predikce.csv')

    #GRAPH
    import matplotlib.pyplot as plt
    # Use datetime for creating date objects for plotting
    import datetime
    # Dates of training values
    months = dataset2.iloc[0:78, 1].values
    days = dataset2.iloc[0:78, 0].values
    years = dataset2.iloc[0:78, 2].values
    months_xgb = dataset4.iloc[0:78, 1].values
    days_xgb = dataset4.iloc[0:78, 0].values
    years_xgb = dataset4.iloc[0:78, 2].values
    months_mfcnb = dataset_mfcnb.iloc[0:54, 1].values
    days_mfcnb = dataset_mfcnb.iloc[0:54, 0].values
    years_mfcnb = dataset_mfcnb.iloc[0:54, 2].values
    # List and then convert to datetime object
    dates_endo = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
    dates_endo = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in dates_endo]

    dates_endo_xgb = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_xgb, months_xgb, days_xgb)]
    dates_endo_xgb = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in dates_endo_xgb]

    dates_endo_mfcnb = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_mfcnb, months_mfcnb, days_mfcnb)]
    dates_endo_mfcnb = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in dates_endo_mfcnb]

    # Dataframe with true values and date
    true_data_endo = pd.DataFrame(data = {'date_endo': dates_endo[0:71], 'actual_endo': dataset2.iloc[0:71, 2+x].values})
    # Dataframe with exogenous predictions
    predictions_data = pd.DataFrame(data = {'date_endo': dates_endo[70:78], 'prediction': dataset2.iloc[70:78, 2+x].values})
    predictions_data_xgb = pd.DataFrame(data = {'date_endo': dates_endo_xgb[70:78], 'prediction': dataset4.iloc[70:78, 2+x].values})
    # Dates of predictions
    months = dataset2.iloc[79:4000, 1].values
    days = dataset2.iloc[79:4000, 0].values
    years = dataset2.iloc[79:4000, 2].values
    months_xgb = dataset4.iloc[79:4000, 1].values
    days_xgb = dataset4.iloc[79:4000, 0].values
    years_xgb = dataset4.iloc[79:4000, 2].values
    months_mfcnb = dataset_mfcnb.iloc[0:54, 1].values
    days_mfcnb = dataset_mfcnb.iloc[0:54, 0].values
    years_mfcnb = dataset_mfcnb.iloc[0:54, 2].values
    # Column of dates
    test_dates_endo = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
    test_dates_endo_xgb = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_xgb, months_xgb, days_xgb)]
    test_dates_endo_mfcnb = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_mfcnb, months_mfcnb, days_mfcnb)]
    # Convert to datetime objects
    test_dates_endo = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in test_dates_endo]
    test_dates_endo_xgb = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in test_dates_endo_xgb]
    test_dates_endo_mfcnb = [datetime.datetime.strptime(date_endo, '%Y-%m-%d') for date_endo in test_dates_endo_mfcnb]
    # Dataframe with predictions and dates
    predictions_data_endo = pd.DataFrame(data = {'date_endo': test_dates_endo, 'prediction_endo': dataset2.iloc[79:4000, 2+x].values})
    predictions_data_endo_xgb = pd.DataFrame(data = {'date_endo': test_dates_endo_xgb, 'prediction_endo': dataset4.iloc[79:4000, 2+x].values})
    predictions_data_endo_cnb = pd.DataFrame(data = {'date_endo': test_dates_endo_mfcnb, 'prediction_endo': dataset_mfcnb.iloc[0:54, 3].values})
    predictions_data_endo_mf = pd.DataFrame(data = {'date_endo': test_dates_endo_mfcnb, 'prediction_endo': dataset_mfcnb.iloc[0:54, 4].values})
    variab = ['Gross domestic product', 'Consumption', 'Export', 'IMPORT', 'PRIBOR', 'Consumption sentiment', 'PMI manufacturing', 'Construction confidence', 'Industrial confidence', 'Wage', 'GDP EA']

    ## EVIEWS
    dataset_ev = pd.read_csv('eviews_forecast.csv') 
    dataset_ev=dataset_ev.iloc[46:89, 0:27].values
    dataset_evnames=['day', 'month', 'year', 'gdp_vecm', 'gdp_fabvarx', 'gdp_fabvar', 'gdp_forecast', 'cons_fabvar', 'cons_favar', 'cons_var', 'cons_forecast', 'im_bvarx', 'im_fabvarx', 'im_forecast', 'infl_bvarx', 'infl_forecast', 'wage_fabvar', 'wage_favar', 'wage_forecast', 'inv_fabvar', 'inv_var', 'inv_forecast', 'lab_vecm', 'lab_forecast', 'ex_bvarx', 'ex_fabvarx', 'ex_forecast']
    dfObjev = pd.DataFrame(dataset_ev, columns=dataset_evnames)
    dfObjev.replace('#N/A', 4)
    dfObjev.to_csv('GDP_eviews.csv', index=False)
    dataset_eviews = pd.read_csv('GDP_eviews.csv')

    import matplotlib.pyplot as plt
    import datetime

    months = dataset_eviews.iloc[0:37, 1].values #43
    days = dataset_eviews.iloc[0:37, 0].values
    years = dataset_eviews.iloc[0:37, 2].values
    months_exo = dataset_eviews.iloc[37:43, 1].values #43
    days_exo = dataset_eviews.iloc[37:43, 0].values
    years_exo = dataset_eviews.iloc[37:43, 2].values

    dates_eviews = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
    dates_eviews = [datetime.datetime.strptime(date_eviews, '%Y-%m-%d') for date_eviews in dates_eviews]
    dates_eviews_exo = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years_exo, months_exo, days_exo)]
    dates_eviews_exo = [datetime.datetime.strptime(date_eviews, '%Y-%m-%d') for date_eviews in dates_eviews_exo]
    eviews_gdp_forecast= pd.DataFrame(data = {'date_eviews': dates_eviews, 'prediction': dataset_eviews.iloc[0:37, 6].values})
    eviews_cons_forecast= pd.DataFrame(data = {'date_eviews': dates_eviews, 'prediction': dataset_eviews.iloc[0:37, 10].values})
    eviews_im_forecast= pd.DataFrame(data = {'date_eviews': dates_eviews, 'prediction': dataset_eviews.iloc[0:37, 13].values})
    eviews_ex_forecast= pd.DataFrame(data = {'date_eviews': dates_eviews, 'prediction': dataset_eviews.iloc[0:37, 26].values})
    eviews_gdp_forecast_exo= pd.DataFrame(data = {'date_eviews': dates_eviews_exo, 'prediction': dataset_eviews.iloc[37:43, 6].values})
    eviews_cons_forecast_exo= pd.DataFrame(data = {'date_eviews': dates_eviews_exo, 'prediction': dataset_eviews.iloc[37:43, 10].values})
    eviews_im_forecast_exo= pd.DataFrame(data = {'date_eviews': dates_eviews_exo, 'prediction': dataset_eviews.iloc[37:43, 13].values})
    eviews_ex_forecast_exo= pd.DataFrame(data = {'date_eviews': dates_eviews_exo, 'prediction': dataset_eviews.iloc[37:43, 26].values})

# AVERAGE FUTURE FORECAST OBTAINED FROM RF AND XGB AS ONE VALUE AND FROM THE EVIEWS AS ANOTHER ONE
    future_gdp=dataset2.iloc[70:76,3].values
    future_gdp[1:6]=((dataset2.iloc[71:76,3].values+dataset4.iloc[71:76,3].values)/2+dataset_eviews.iloc[38:43, 6].values)/2
    future_con=dataset2.iloc[70:76,4].values
    future_con[1:6]=((dataset2.iloc[71:76,4].values+dataset4.iloc[71:76,4].values)/2+dataset_eviews.iloc[38:43, 10].values)/2
    future_exp=dataset2.iloc[70:76,5].values
    future_exp[1:6]=((dataset2.iloc[71:76,5].values+dataset4.iloc[71:76,5].values)/2+dataset_eviews.iloc[38:43, 26].values)/2
    future_imp=dataset2.iloc[70:76,6].values
    future_imp[1:6]=((dataset2.iloc[71:76,6].values+dataset4.iloc[71:76,6].values)/2+dataset_eviews.iloc[38:43, 13].values)/2
    
    if scenario !=6:
        fig.add_trace(go.Scatter(name="Data",x=true_data_endo['date_endo'], y=true_data_endo['actual_endo'], mode = 'lines', line = dict(color='black', width=3), showlegend=False), row=x,col=1)

    if scenario==6: #THE INTEREST RATE
        fig.add_trace(go.Scatter(name="Data",x=true_data_endo['date_endo'], y=true_data_endo['actual_endo'], mode = 'lines', marker = dict(color='black'), showlegend=False), row=x,col=1)

        fig.add_trace(go.Scatter(name="RF",x=predictions_data_endo['date_endo'], y=predictions_data_endo['prediction_endo'], mode = 'markers', marker=dict(color='Pink'), showlegend=False), row=x,col=1)

        fig.add_trace(go.Scatter(name="XGB",x=predictions_data_endo_xgb['date_endo'], y=predictions_data_endo_xgb['prediction_endo'], mode = 'markers', marker=dict(color='Purple'), showlegend=False), row=x,col=1)
    
    #THE GDP GRAPH
    if scenario==1: 
        ex_forecast= pd.DataFrame(data = {'date_endo': dates_endo[70:76], 'prediction': future_gdp})
        fig.add_trace(go.Scatter(x=ex_forecast['date_endo'], y=ex_forecast['prediction'], mode = 'lines', marker=dict(color='crimson'), showlegend=False), row=x,col=1)
        fig.add_trace(go.Scatter(name="XGB_gdp",x=predictions_data_endo_xgb['date_endo'], y=predictions_data_endo_xgb['prediction_endo'], mode = 'markers', marker=dict(color='rgba(128,0,128, 0.8)', size=7,line=dict(color='darkblue', width=1))), row=x,col=1) #red
        fig.add_trace(go.Scatter(name="VAR_gdp",x=eviews_gdp_forecast['date_eviews'], y= eviews_gdp_forecast['prediction'], mode = 'markers', marker=dict(color='rgba(105,105,105, 0.8)', size=7, line=dict(color='darkblue', width=1))), row=x,col=1) #orange 
        fig.add_trace(go.Scatter(name="RF_gdp",x=predictions_data_endo['date_endo'], y=predictions_data_endo['prediction_endo'], mode = 'markers', marker=dict(color='rgba(255,192,203, 0.8)',size=7, line=dict(color='darkblue', width=1))), row=x,col=1)

        fig.add_trace(go.Scatter(name="cnb",x=predictions_data_endo_cnb['date_endo'], y=predictions_data_endo_cnb['prediction_endo'], mode = 'markers', marker=dict(color='rgba(0,0,205, 0.8)', size=7, line=dict(color='darkblue', width=1))), row=x,col=1) #mediumblue

        fig.add_trace(go.Scatter(name="mf",x=predictions_data_endo_mf['date_endo'], y=predictions_data_endo_mf['prediction_endo'], mode = 'markers', marker=dict(color='rgba(0,255,255, 0.8)', size=7, line=dict(color='darkblue', width=1))), row=x,col=1) #aqua
        
    #THE CONSUMPTION GRAPH
    if scenario==2:
        fig.add_trace(go.Scatter(name="XGB_con",x=predictions_data_endo_xgb['date_endo'], y=predictions_data_endo_xgb['prediction_endo'], mode = 'markers', marker=dict(color='rgba(128,0,128, 0.7)', size=7,line=dict(color='darkblue', width=1))), row=x,col=1)
        fig.add_trace(go.Scatter(name="VAR_con",x=eviews_cons_forecast['date_eviews'], y= eviews_cons_forecast['prediction'], mode = 'markers', marker=dict(color='rgba(105,105,105, 0.7)', size=7, line=dict(color='darkblue', width=1))), row=x,col=1)
        fig.add_trace(go.Scatter(name="RF_con",x=predictions_data_endo['date_endo'], y=predictions_data_endo['prediction_endo'], mode = 'markers', marker=dict(color='rgba(255,192,203, 0.7)',size=7, line=dict(color='darkblue', width=1))), row=x,col=1)
        ex_forecast= pd.DataFrame(data = {'date_endo': dates_endo[70:76], 'prediction': future_con})

        fig.add_trace(go.Scatter(x=ex_forecast['date_endo'], y=ex_forecast['prediction'], mode = 'lines', marker=dict(color='crimson'), showlegend=False), row=x,col=1)

    #THE EXPORT GRAPH
    if scenario==3:
        fig.add_trace(go.Scatter(name="XGB_ex",x=predictions_data_endo_xgb['date_endo'], y=predictions_data_endo_xgb['prediction_endo'], mode = 'markers', marker=dict(color='rgba(128,0,128, 0.7)', size=7,line=dict(color='darkblue', width=1))), row=x,col=1)
        fig.add_trace(go.Scatter(name="VAR_ex",x=eviews_ex_forecast['date_eviews'], y= eviews_ex_forecast['prediction'], mode = 'markers', marker=dict(color='rgba(105,105,105, 0.7)', size=7, line=dict(color='darkblue', width=1))), row=x,col=1)
        fig.add_trace(go.Scatter(name="RF_ex",x=predictions_data_endo['date_endo'], y=predictions_data_endo['prediction_endo'], mode = 'markers', marker=dict(color='rgba(255,192,203, 0.7)',size=7, line=dict(color='darkblue', width=1))), row=x,col=1)
        ex_forecast= pd.DataFrame(data = {'date_endo': dates_endo[70:76], 'prediction': future_exp})
        fig.add_trace(go.Scatter(x=ex_forecast['date_endo'], y=ex_forecast['prediction'], mode = 'lines', marker=dict(color='crimson'), showlegend=False), row=x,col=1)

    #THE IMPORT GRAPH
    if scenario==4:
        fig.add_trace(go.Scatter(name="XGB_im",x=predictions_data_endo_xgb['date_endo'], y=predictions_data_endo_xgb['prediction_endo'], mode = 'markers', marker=dict(color='rgba(128,0,128, 0.7)', size=7,line=dict(color='darkblue', width=1))), row=x,col=1)
        fig.add_trace(go.Scatter(name="VAR_im",x=eviews_im_forecast['date_eviews'], y= eviews_im_forecast['prediction'], mode = 'markers', marker=dict(color='rgba(105,105,105, 0.7)', size=7, line=dict(color='darkblue', width=1))), row=x,col=1)
        fig.add_trace(go.Scatter(name="RF_im",x=predictions_data_endo['date_endo'], y=predictions_data_endo['prediction_endo'], mode = 'markers', marker=dict(color='rgba(255,192,203, 0.7)',size=7, line=dict(color='darkblue', width=1))), row=x,col=1)
        ex_forecast= pd.DataFrame(data = {'date_endo': dates_endo[70:76], 'prediction': future_imp})
        fig.add_trace(go.Scatter(x=ex_forecast['date_endo'], y=ex_forecast['prediction'], mode = 'lines', marker=dict(color='crimson'), showlegend=False), row=x,col=1)

    

fig.update_layout(legend=dict(orientation="h",
yanchor="bottom",
y=-0.06,
xanchor="right",
x=1
))
#fig.update_layout(
   # margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(
    autosize=False,
    width=950,
    height=1000)   
fig.update_xaxes(matches='x')
#ax = plt.gca()
#ax.set_facecolor('white')
fig.show()

#if you want to save it to your chart-studio:
#py.plot(fig, filename='random_forest_subplots1_new', auto_open=True)


